# Сustomer response models

In [1]:
!pip3 install --upgrade nbformat


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px 


import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats as stats
import re
import traceback
import string

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')


In [3]:
data = pd.read_csv("vitrina_final1.csv")

In [4]:
data

,Телефон_new,Выручка_bought,Абсолютная маржа_bought,Маржа_bought,Средняя маржа_bought,Средний чек_bought,Кол-во чеков_bought,Кол-во товаров_bought,Среднее кол-во товаров в чеке_bought,Кол-во уникальных клиентов_bought,...,ПОДГУЗНИКИ выручка,ДЕТСКОЕ ПИТАНИЕ выручка,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА выручка,ТОВАРЫ ДЛЯ ЖИВОТНЫХ выручка,КГТ выручка,ОДЕЖДА выручка,ИГРУШКИт выручка,ППКП выручка,ИНОЕ выручка,target
0,55575453-56535648535679,4875,846,0.17,70.50,380,12,16,6,1,...,390.0,2047.5,0.0,0.0,0.0,1218.75,1218.75,2437.5,0.0,0
1,55574950-57515657535772,3325,2176,0.65,2176.00,3325,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.00,3325.00,0.0,0.0,0
2,55574854-56495552515179,528,204,0.39,51.00,132,4,4,4,1,...,0.0,0.0,0.0,0.0,0.0,0.00,264.00,264.0,0.0,0
3,55574851-53495753545674,2962,654,0.22,109.00,494,6,6,6,1,...,0.0,0.0,0.0,0.0,0.0,503.54,2458.46,0.0,0.0,0
4,55574948-52515357485779,2196,759,0.35,189.75,366,4,6,3,1,...,0.0,0.0,0.0,0.0,0.0,2196.00,0.00,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83609,55574851-50535648495271,886,296,0.33,296.00,443,1,2,2,1,...,0.0,0.0,0.0,0.0,0.0,886.00,0.00,0.0,0.0,1
83610,55575056-49565452495078,10390,2369,0.23,2369.00,10390,1,1,1,1,...,0.0,0.0,0.0,0.0,10390.0,0.00,0.00,0.0,0.0,1
83611,55575050-50505549555678,375,38,0.10,38.00,375,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,375.0,0.0,1
83612,55574948-52535051534878,320,57,0.18,57.00,320,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,320.0,0.0,1


In [5]:
data.drop(['Телефон_new'], axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83614 entries, 0 to 83613
Data columns (total 44 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Выручка_bought                         83614 non-null  int64  
 1   Абсолютная маржа_bought                83614 non-null  int64  
 2   Маржа_bought                           83614 non-null  float64
 3   Средняя маржа_bought                   83614 non-null  float64
 4   Средний чек_bought                     83614 non-null  int64  
 5   Кол-во чеков_bought                    83614 non-null  int64  
 6   Кол-во товаров_bought                  83614 non-null  int64  
 7   Среднее кол-во товаров в  чеке_bought  83614 non-null  int64  
 8   Кол-во уникальных клиентов_bought      83614 non-null  int64  
 9   Ценность клиента_bought                83614 non-null  int64  
 10  Центральный ф.о.                       83614 non-null  float64
 11  Пр

In [6]:
data.head()

,Выручка_bought,Абсолютная маржа_bought,Маржа_bought,Средняя маржа_bought,Средний чек_bought,Кол-во чеков_bought,Кол-во товаров_bought,Среднее кол-во товаров в чеке_bought,Кол-во уникальных клиентов_bought,Ценность клиента_bought,...,ПОДГУЗНИКИ выручка,ДЕТСКОЕ ПИТАНИЕ выручка,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА выручка,ТОВАРЫ ДЛЯ ЖИВОТНЫХ выручка,КГТ выручка,ОДЕЖДА выручка,ИГРУШКИт выручка,ППКП выручка,ИНОЕ выручка,target
0,4875,846,0.17,70.50,380,12,16,6,1,846,...,390.0,2047.5,0.0,0.0,0.0,1218.75,1218.75,2437.5,0.0,0
1,3325,2176,0.65,2176.00,3325,1,1,1,1,2176,...,0.0,0.0,0.0,0.0,0.0,0.00,3325.00,0.0,0.0,0
2,528,204,0.39,51.00,132,4,4,4,1,204,...,0.0,0.0,0.0,0.0,0.0,0.00,264.00,264.0,0.0,0
3,2962,654,0.22,109.00,494,6,6,6,1,654,...,0.0,0.0,0.0,0.0,0.0,503.54,2458.46,0.0,0.0,0
4,2196,759,0.35,189.75,366,4,6,3,1,759,...,0.0,0.0,0.0,0.0,0.0,2196.00,0.00,0.0,0.0,0


In [7]:
data['target'].sum() / len(data['target']) * 100

54.43586002344105

#### 51.8% of all clients are coming back to the shop


In [8]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler

In [9]:
X = data.drop(columns=['target'])
y = data['target']

# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

## Patterns selection


#### Information Value (IV)

In [10]:
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV': iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df, iv)

In [11]:
final_iv, IV = data_vars(X_train.copy(), y_train.copy())

In [12]:
IV.head()

,VAR_NAME,IV
0,Абсолютная маржа_bought,0.032666
1,Восточно-Сибирский ф.о.,0.000000
2,Выручка_bought,0.048013
3,ДЕТСКОЕ ПИТАНИЕ выручка,0.025518
4,Дальневосточный ф.о.,0.000000


#### Selecting patterns with  IV >= 0.01

In [13]:
important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

In [14]:
IV[(IV['IV'] >= 0.01)]

,VAR_NAME,IV
0,Абсолютная маржа_bought,0.032666
2,Выручка_bought,0.048013
3,ДЕТСКОЕ ПИТАНИЕ выручка,0.025518
14,КОСМЕТИКА/ГИГИЕНА выручка,0.018881
16,Кол-во товаров_bought,0.088781
18,Кол-во чеков_bought,0.092893
23,ОДЕЖДА выручка,0.024764
24,ПОДГУЗНИКИ выручка,0.015629
26,ППКП выручка,0.020786
32,Среднее кол-во товаров в чеке_bought,0.038816


In [15]:
pip install ipykernel


Note: you may need to restart the kernel to use updated packages.


In [18]:
colors = ["#016450", "#02818a", "#3690c0", "#67a9cf", "#a6bddb", "#d0d1e6", "#ece2f0", "#fff7fb"]
from sklearn.inspection import permutation_importance
IV=IV.sort_values('IV', ascending =True)

fig = px.bar(IV[(IV['IV'] >= 0.01)], x='IV', y='VAR_NAME',
             orientation='h', color_discrete_sequence=px.colors.sequential.Agsunset, labels={
                     "VAR_NAME": " "
                   })

fig.update_layout(xaxis_title="Information Value")  # Update the x-axis label

fig.show()

In [19]:
print(important_columns)

['Абсолютная маржа_bought' 'Выручка_bought' 'ДЕТСКОЕ ПИТАНИЕ выручка'
 'КОСМЕТИКА/ГИГИЕНА выручка' 'Кол-во товаров_bought' 'Кол-во чеков_bought'
 'ОДЕЖДА выручка' 'ПОДГУЗНИКИ выручка' 'ППКП выручка'
 'Среднее кол-во товаров в  чеке_bought' 'Средний чек_bought'
 'ТЕКСТИЛЬ, ТРИКОТАЖ выручка' 'ТОВАРЫ ДЛЯ КОРМЛЕНИЯ выручка'
 'Ценность клиента_bought']


In [20]:
X_train.shape

(50168, 43)

In [21]:
X_train[important_columns].shape

(50168, 14)

In [22]:
X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]

# RandomForestClassifier

In [23]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
clf_forest = RandomForestClassifier(random_state=609)
clf_forest.fit(X_train, y_train)
train_pred = clf_forest.predict(X_train)
test_pred = clf_forest.predict(X_test)

In [26]:
fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

print('ROC_AUC Train:', "%.2f" % roc_auc_score(y_train, train_pred))
print('ROC_AUC Test:', "%.2f" % roc_auc_score(y_test, test_pred))

print('F1 Train:', "%.2f" % f1_score(y_train, train_pred))
print('F1 Test:', "%.2f" % f1_score(y_test, test_pred))

print('GINI Train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
print('GINI Test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))

ROC_AUC Train: 0.96
ROC_AUC Test: 0.71
F1 Train: 0.96
F1 Test: 0.74
GINI Train: 0.92
GINI Test: 0.42


In [27]:
test_pred

array([1, 1, 0, ..., 1, 1, 1])

### Selecting hyperparameters

In [28]:
n_estimators = [20, 50, 100, 200]

In [29]:
F1_train = []
F1_test = []
GINI_train = []
GINI_test = []
ROC_AUC_train = []
ROC_AUC_test = []

for estimators in n_estimators:

    clf_forest = RandomForestClassifier(random_state=0, n_estimators = estimators)
    clf_forest.fit(X_train, y_train)
    train_pred = clf_forest.predict(X_train)
    test_pred = clf_forest.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    print('Estimators:', estimators)
    print('ROC_AUC train:', "%.2f" % roc_auc_score(y_train, train_pred))
    print('ROC_AUC test:', "%.2f" % roc_auc_score(y_test, test_pred))

    print('F1 train:', "%.2f" % f1_score(y_train, train_pred))
    print('F1 test:', "%.2f" % f1_score(y_test, test_pred))

    print('GINI train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
    print('GINI test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))
    print()
    
    F1_train.append(f1_score(y_train, train_pred))
    GINI_train.append((2 * auc(fpr_train, tpr_train) - 1))
    ROC_AUC_train.append(roc_auc_score(y_train, train_pred))
    
    
    F1_test.append(f1_score(y_test, test_pred))
    GINI_test.append((2 * auc(fpr_test, tpr_test) - 1))
    ROC_AUC_test.append(roc_auc_score(y_test, test_pred))

Estimators: 20
ROC_AUC train: 0.95
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.73
GINI train: 0.91
GINI test: 0.41

Estimators: 50
ROC_AUC train: 0.96
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.74
GINI train: 0.91
GINI test: 0.42

Estimators: 100
ROC_AUC train: 0.96
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.74
GINI train: 0.92
GINI test: 0.42

Estimators: 200
ROC_AUC train: 0.96
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.75
GINI train: 0.92
GINI test: 0.42



In [25]:
import plotly.express as px 
colors = ["#016450", "#02818a", "#3690c0", "#67a9cf", "#a6bddb", "#d0d1e6", "#ece2f0", "#fff7fb"]

lines = pd.DataFrame({
"F1 Train" : F1_train,
"F1 Test" : F1_test,
"GINI Train" : GINI_train,
"GINI Test" : GINI_test,
"ROC_AUC Train" : ROC_AUC_train,
"ROC_AUC Test" : ROC_AUC_test,    
}, index=n_estimators)

fig = px.line(lines, x=lines.index, y=lines.columns,title='Dependence of F-score & GINI score on number of estimators',color_discrete_sequence = px.colors.sequential.algae, labels={
                     "index": "Number of estimators",
                     "value": "Error Value"})
fig.update_traces(line=dict(width=3))
fig.update_layout(xaxis_range=[min(n_estimators)-10,max(n_estimators)+10])

fig.show()

In [26]:
best_n_estimators = 100

#### max depth


In [27]:
max_depth = [20, 50, 100, 150]

In [28]:
F1_train = []
F1_test = []

GINI_train = []
GINI_test = []

ROC_AUC_train = []
ROC_AUC_test = []

for depth in max_depth:

    clf_forest = RandomForestClassifier(random_state=0, n_estimators = best_n_estimators, max_depth = depth)
    clf_forest.fit(X_train, y_train)
    train_pred = clf_forest.predict(X_train)
    test_pred = clf_forest.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    print('Max depth:', depth)
    print('ROC_AUC train:', "%.2f" % roc_auc_score(y_train, train_pred))
    print('ROC_AUC test:', "%.2f" % roc_auc_score(y_test, test_pred))

    print('F1 train:', "%.2f" % f1_score(y_train, train_pred))
    print('F1 test:', "%.2f" % f1_score(y_test, test_pred))

    print('GINI train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
    print('GINI test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1)) 
    print()
    
    F1_train.append(f1_score(y_train, train_pred))
    GINI_train.append((2 * auc(fpr_train, tpr_train) - 1))
    ROC_AUC_train.append(roc_auc_score(y_train, train_pred))
    
    
    F1_test.append(f1_score(y_test, test_pred))
    GINI_test.append((2 * auc(fpr_test, tpr_test) - 1))
    ROC_AUC_test.append(roc_auc_score(y_test, test_pred))
    

Max depth: 20
ROC_AUC train: 0.88
ROC_AUC test: 0.68
F1 train: 0.87
F1 test: 0.68
GINI train: 0.75
GINI test: 0.36

Max depth: 50
ROC_AUC train: 0.96
ROC_AUC test: 0.70
F1 train: 0.96
F1 test: 0.71
GINI train: 0.91
GINI test: 0.40

Max depth: 100
ROC_AUC train: 0.96
ROC_AUC test: 0.70
F1 train: 0.96
F1 test: 0.71
GINI train: 0.91
GINI test: 0.40

Max depth: 150
ROC_AUC train: 0.96
ROC_AUC test: 0.70
F1 train: 0.96
F1 test: 0.71
GINI train: 0.91
GINI test: 0.40



In [29]:
import plotly.express as px 
colors = ["#016450", "#02818a", "#3690c0", "#67a9cf", "#a6bddb", "#d0d1e6", "#ece2f0", "#fff7fb"]

lines = pd.DataFrame({
"F1 Train" : F1_train,
"F1 Test" : F1_test,
"GINI Train" : GINI_train,
"GINI Test" : GINI_test,
"ROC_AUC Train" : ROC_AUC_train,
"ROC_AUC Test" : ROC_AUC_test,    
}, index=max_depth)

fig = px.line(lines, x=lines.index, y=lines.columns,title='Dependence of F-score, GINI score on max_depth',color_discrete_sequence = colors, labels={
                     "index": "Max_depth",
                     "value": "Error Value"})
fig.update_traces(line=dict(width=3))
fig.update_layout(xaxis_range=[min(max_depth)-10,max(max_depth)+10])

fig.show()

In [30]:
best_max_depth = 50

### Final Random Forest model with chosen hyperparameters


In [31]:
clf_forest = RandomForestClassifier(random_state=0, n_estimators = best_n_estimators, max_depth = best_max_depth)
clf_forest.fit(X_train, y_train)
train_pred = clf_forest.predict(X_train)
test_pred = clf_forest.predict(X_test)

In [32]:
fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

print('ROC_AUC Train:', "%.2f" % roc_auc_score(y_train, train_pred))
print('ROC_AUC Test:', "%.2f" % roc_auc_score(y_test, test_pred))

print('F1 Train:', "%.2f" % f1_score(y_train, train_pred))
print('F1 Test:', "%.2f" % f1_score(y_test, test_pred))

print('GINI Train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
print('GINI Test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))

ROC_AUC Train: 0.96
ROC_AUC Test: 0.70
F1 Train: 0.96
F1 Test: 0.71
GINI Train: 0.91
GINI Test: 0.40


In [31]:
# colors = ["#016450", "#02818a", "#3690c0", "#67a9cf", "#a6bddb", "#d0d1e6", "#ece2f0", "#fff7fb"]
from sklearn.inspection import permutation_importance
sort = clf_forest.feature_importances_.argsort()

fig = px.bar(x=clf_forest.feature_importances_[sort], y=data.columns[sort],
             orientation='h', color_discrete_sequence=px.colors.sequential.algae_r, labels={
                     "y": " "
                   })

fig.update_layout(xaxis_title="Feature Importance")  # Update the x-axis label

fig.show()

# DecisionTreeClassifier

In [32]:
from sklearn.tree import DecisionTreeClassifier

In [33]:
clf_tree = DecisionTreeClassifier(random_state=0)
clf_tree.fit(X_train, y_train)
train_pred = clf_tree.predict(X_train)
test_pred = clf_tree.predict(X_test)

In [35]:
fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

print('ROC_AUC Train:', "%.2f" % roc_auc_score(y_train, train_pred))
print('ROC_AUC Test:', "%.2f" % roc_auc_score(y_test, test_pred))

print('F1 Train:', "%.2f" % f1_score(y_train, train_pred))
print('F1 Test:', "%.2f" % f1_score(y_test, test_pred))

print('GINI Train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
print('GINI Test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))

ROC_AUC Train: 0.96
ROC_AUC Test: 0.69
F1 Train: 0.96
F1 Test: 0.71
GINI Train: 0.92
GINI Test: 0.39


### Selecting hyperparameters

In [36]:
max_depth = [20, 50, 100, 150, 200]

In [37]:
F1_train = []
F1_test = []

GINI_train = []
GINI_test = []

ROC_AUC_train = []
ROC_AUC_test = []

for depth in max_depth:

    clf_tree = DecisionTreeClassifier(random_state=0, max_depth = depth)
    clf_tree.fit(X_train, y_train)
    train_pred = clf_tree.predict(X_train)
    test_pred = clf_tree.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    print('Max depth:', depth)
    print('ROC_AUC train:', "%.2f" % roc_auc_score(y_train, train_pred))
    print('ROC_AUC test:', "%.2f" % roc_auc_score(y_test, test_pred))

    print('F1 train:', "%.2f" % f1_score(y_train, train_pred))
    print('F1 test:', "%.2f" % f1_score(y_test, test_pred))

    print('GINI train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
    print('GINI test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))
    print()
    
    F1_train.append(f1_score(y_train, train_pred))
    GINI_train.append((2 * auc(fpr_train, tpr_train) - 1))
    ROC_AUC_train.append(roc_auc_score(y_train, train_pred))
    
    
    F1_test.append(f1_score(y_test, test_pred))
    GINI_test.append((2 * auc(fpr_test, tpr_test) - 1))
    ROC_AUC_test.append(roc_auc_score(y_test, test_pred))
    

Max depth: 20
ROC_AUC train: 0.78
ROC_AUC test: 0.64
F1 train: 0.76
F1 test: 0.63
GINI train: 0.56
GINI test: 0.29

Max depth: 50
ROC_AUC train: 0.94
ROC_AUC test: 0.69
F1 train: 0.95
F1 test: 0.71
GINI train: 0.88
GINI test: 0.37

Max depth: 100
ROC_AUC train: 0.96
ROC_AUC test: 0.69
F1 train: 0.96
F1 test: 0.71
GINI train: 0.92
GINI test: 0.39

Max depth: 150
ROC_AUC train: 0.96
ROC_AUC test: 0.69
F1 train: 0.96
F1 test: 0.71
GINI train: 0.92
GINI test: 0.39

Max depth: 200
ROC_AUC train: 0.96
ROC_AUC test: 0.69
F1 train: 0.96
F1 test: 0.71
GINI train: 0.92
GINI test: 0.39



In [38]:

lines = pd.DataFrame({
"F1 Train" : F1_train,
"F1 Test" : F1_test,
"GINI Train" : GINI_train,
"GINI Test" : GINI_test,
"ROC_AUC Train" : ROC_AUC_train,
"ROC_AUC Test" : ROC_AUC_test,
}, index=max_depth)

fig = px.line(lines, x=lines.index, y=lines.columns,title='Dependence of F-score & GINI score on max_depth',color_discrete_sequence = colors, labels={
                     "index": "Max depth",
                     "value": "Error Value"})
fig.update_traces(line=dict(width=3))
fig.update_layout(xaxis_range=[min(max_depth)-10,max(max_depth)+10])

fig.show()

In [39]:
best_max_depth = 50


In [40]:
min_samples_split = [2, 5, 7]

In [41]:
F1_train = []
F1_test = []

GINI_train = []
GINI_test = []
ROC_AUC_train = []
ROC_AUC_test = []

for sample_split in min_samples_split:

    clf_tree = DecisionTreeClassifier(random_state=0, max_depth = best_max_depth, min_samples_split = sample_split)
    clf_tree.fit(X_train, y_train)
    train_pred = clf_tree.predict(X_train)
    test_pred = clf_tree.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    print('min_samples_split:', sample_split)
    print('ROC_AUC train:', "%.2f" % roc_auc_score(y_train, train_pred))
    print('ROC_AUC test:', "%.2f" % roc_auc_score(y_test, test_pred))

    print('F1 train:', "%.2f" % f1_score(y_train, train_pred))
    print('F1 test:', "%.2f" % f1_score(y_test, test_pred))

    print('GINI train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
    print('GINI test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))
    print()
    
    F1_train.append(f1_score(y_train, train_pred))
    GINI_train.append((2 * auc(fpr_train, tpr_train) - 1))
    ROC_AUC_train.append(roc_auc_score(y_train, train_pred))
    
    
    F1_test.append(f1_score(y_test, test_pred))
    GINI_test.append((2 * auc(fpr_test, tpr_test) - 1))
    ROC_AUC_test.append(roc_auc_score(y_test, test_pred))
    

min_samples_split: 2
ROC_AUC train: 0.94
ROC_AUC test: 0.69
F1 train: 0.95
F1 test: 0.71
GINI train: 0.88
GINI test: 0.37

min_samples_split: 5
ROC_AUC train: 0.91
ROC_AUC test: 0.68
F1 train: 0.92
F1 test: 0.70
GINI train: 0.82
GINI test: 0.36

min_samples_split: 7
ROC_AUC train: 0.89
ROC_AUC test: 0.67
F1 train: 0.90
F1 test: 0.69
GINI train: 0.78
GINI test: 0.34



In [42]:
lines = pd.DataFrame({
"F1 Train" : F1_train,
"F1 Test" : F1_test,
"GINI Train" : GINI_train,
"GINI Test" : GINI_test,
"ROC_AUC Train" : ROC_AUC_train,
"ROC_AUC Test" : ROC_AUC_test,
}, index=min_samples_split)

fig = px.line(lines, x=lines.index, y=lines.columns,title='Dependence of F-score & GINI score on min_samples_split',color_discrete_sequence = colors, labels={
                     "index": "Min_samples_split",
                     "value": "Error Value"})
fig.update_traces(line=dict(width=3))
fig.update_layout(xaxis_range=[min(min_samples_split)-0.5,max(min_samples_split)+0.5])

fig.show()

In [43]:
best_min_samples_split = 2

### Decision Tree with chosen hyperparameters

In [44]:
clf_tree = DecisionTreeClassifier(random_state=0, max_depth = best_max_depth, min_samples_split = best_min_samples_split)
clf_tree.fit(X_train, y_train)
train_pred = clf_tree.predict(X_train)
test_pred = clf_tree.predict(X_test)

In [45]:
fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

print('ROC_AUC train:', "%.2f" % roc_auc_score(y_train, train_pred))
print('ROC_AUC test:', "%.2f" % roc_auc_score(y_test, test_pred))

print('F1 train:', "%.2f" % f1_score(y_train, train_pred))
print('F1 test:', "%.2f" % f1_score(y_test, test_pred))

print('GINI train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
print('GINI test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))

ROC_AUC train: 0.94
ROC_AUC test: 0.69
F1 train: 0.95
F1 test: 0.71
GINI train: 0.88
GINI test: 0.37


In [46]:
colors = ["#016450", "#02818a", "#3690c0", "#67a9cf", "#a6bddb", "#d0d1e6", "#ece2f0", "#fff7fb"]
from sklearn.inspection import permutation_importance
sort = clf_tree.feature_importances_.argsort()

fig = px.bar(x=clf_tree.feature_importances_[sort], y=data.columns[sort],
             orientation='h', color_discrete_sequence=colors, labels={
                     "y": " "
                   })

fig.update_layout(xaxis_title="Feature Importance")  # Update the x-axis label

fig.show()

# Support Vector Machine

In [47]:
fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

print('ROC_AUC Train:', "%.2f" % roc_auc_score(y_train, train_pred))
print('ROC_AUC Test:', "%.2f" % roc_auc_score(y_test, test_pred))

print('F1 Train:', "%.2f" % f1_score(y_train, train_pred))
print('F1 Test:', "%.2f" % f1_score(y_test, test_pred))

print('GINI Train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
print('GINI Test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))

ROC_AUC Train: 0.94
ROC_AUC Test: 0.69
F1 Train: 0.95
F1 Test: 0.71
GINI Train: 0.88
GINI Test: 0.37


In [48]:
print('ROC_AUC Train:', "%.2f" % roc_auc_score(y_train, train_pred))
print('ROC_AUC Test:', "%.2f" % roc_auc_score(y_test, test_pred))

print('F1 Train:', "%.2f" % f1_score(y_train, train_pred))
print('F1 Test:', "%.2f" % f1_score(y_test, test_pred))

print('GINI Train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
print('GINI Test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))

ROC_AUC Train: 0.94
ROC_AUC Test: 0.69
F1 Train: 0.95
F1 Test: 0.71
GINI Train: 0.88
GINI Test: 0.37


# XGBoost 


In [49]:
#!pip3 install xgboost

In [50]:
from numpy import loadtxt
from xgboost import XGBClassifier

In [51]:
xgboost = XGBClassifier()
xgboost.fit(X_train, y_train)
  
train_pred = xgboost.predict(X_train)
test_pred = xgboost.predict(X_test)

In [52]:
fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

print('ROC_AUC Train:', "%.2f" % roc_auc_score(y_train, train_pred))
print('ROC_AUC Test:', "%.2f" % roc_auc_score(y_test, test_pred))

print('F1 Train:', "%.2f" % f1_score(y_train, train_pred))
print('F1 Test:', "%.2f" % f1_score(y_test, test_pred))

print('GINI Train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
print('GINI Test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))

ROC_AUC Train: 0.72
ROC_AUC Test: 0.64
F1 Train: 0.75
F1 Test: 0.68
GINI Train: 0.44
GINI Test: 0.29


### Selecting hyperparameters

In [53]:
max_depth = [20, 50, 70, 100, 150, 200]

In [54]:
F1_train = []
F1_test = []

GINI_train = []
GINI_test = []
ROC_AUC_train = []
ROC_AUC_test = []

for depth in max_depth:

    xgboost = XGBClassifier(random_state=0, max_depth = depth,)
    xgboost.fit(X_train, y_train)
    train_pred = xgboost.predict(X_train)
    test_pred = xgboost.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    print('Max depth:', depth)
    print('ROC_AUC train:', "%.2f" % roc_auc_score(y_train, train_pred))
    print('ROC_AUC test:', "%.2f" % roc_auc_score(y_test, test_pred))

    print('F1 train:', "%.2f" % f1_score(y_train, train_pred))
    print('F1 test:', "%.2f" % f1_score(y_test, test_pred))

    print('GINI train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
    print('GINI test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))
    print()
    
    F1_train.append(f1_score(y_train, train_pred))
    GINI_train.append((2 * auc(fpr_train, tpr_train) - 1))
    ROC_AUC_train.append(roc_auc_score(y_train, train_pred))
    
    
    F1_test.append(f1_score(y_test, test_pred))
    GINI_test.append((2 * auc(fpr_test, tpr_test) - 1))
    ROC_AUC_test.append(roc_auc_score(y_test, test_pred))
    

Max depth: 20
ROC_AUC train: 0.94
ROC_AUC test: 0.70
F1 train: 0.94
F1 test: 0.73
GINI train: 0.87
GINI test: 0.40

Max depth: 50
ROC_AUC train: 0.95
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.74
GINI train: 0.91
GINI test: 0.41

Max depth: 70
ROC_AUC train: 0.96
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.74
GINI train: 0.91
GINI test: 0.41

Max depth: 100
ROC_AUC train: 0.96
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.74
GINI train: 0.91
GINI test: 0.41

Max depth: 150
ROC_AUC train: 0.96
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.74
GINI train: 0.91
GINI test: 0.41

Max depth: 200
ROC_AUC train: 0.96
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.74
GINI train: 0.92
GINI test: 0.41



In [55]:
lines = pd.DataFrame({
"F1 Train" : F1_train,
"F1 Test" : F1_test,
"GINI Train" : GINI_train,
"GINI Test" : GINI_test,
"ROC_AUC Train" : ROC_AUC_train,
"ROC_AUC Test" : ROC_AUC_test,
}, index=max_depth)

fig = px.line(lines, x=lines.index, y=lines.columns,title='Dependence of F-score & GINI score on max_depth',color_discrete_sequence = colors, labels={
                     "index": "Max depth",
                     "value": "Error Value"})
fig.update_traces(line=dict(width=3))
fig.update_layout(xaxis_range=[min(max_depth)-10,max(max_depth)+10])

fig.show()

In [56]:
best_max_depth = 70

In [57]:
n_estimators = [50, 100, 150, 200]

In [58]:
F1_train = []
F1_test = []

GINI_train = []
GINI_test = []
ROC_AUC_train = []
ROC_AUC_test = []

for estimators in n_estimators:

    xgboost = XGBClassifier(random_state=0, max_depth = best_max_depth, n_estimators = estimators)
    xgboost.fit(X_train, y_train)
    train_pred = xgboost.predict(X_train)
    test_pred = xgboost.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    print('Estimators:', estimators)
    print('ROC_AUC train:', "%.2f" % roc_auc_score(y_train, train_pred))
    print('ROC_AUC test:', "%.2f" % roc_auc_score(y_test, test_pred))

    print('F1 train:', "%.2f" % f1_score(y_train, train_pred))
    print('F1 test:', "%.2f" % f1_score(y_test, test_pred))

    print('GINI train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
    print('GINI test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))
    print()
    
    F1_train.append(f1_score(y_train, train_pred))
    GINI_train.append((2 * auc(fpr_train, tpr_train) - 1))
    ROC_AUC_train.append(roc_auc_score(y_train, train_pred))
    
    
    F1_test.append(f1_score(y_test, test_pred))
    GINI_test.append((2 * auc(fpr_test, tpr_test) - 1))
    ROC_AUC_test.append(roc_auc_score(y_test, test_pred))
    

Estimators: 50
ROC_AUC train: 0.95
ROC_AUC test: 0.70
F1 train: 0.96
F1 test: 0.74
GINI train: 0.90
GINI test: 0.41

Estimators: 100
ROC_AUC train: 0.96
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.74
GINI train: 0.91
GINI test: 0.41

Estimators: 150
ROC_AUC train: 0.96
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.74
GINI train: 0.91
GINI test: 0.41

Estimators: 200
ROC_AUC train: 0.96
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.74
GINI train: 0.91
GINI test: 0.41



In [64]:
lines = pd.DataFrame({
"F1 Train" : F1_train,
"F1 Test" : F1_test,
"GINI Train" : GINI_train,
"GINI Test" : GINI_test,
"ROC_AUC Train" : ROC_AUC_train,
"ROC_AUC Test" : ROC_AUC_test,
}, index=n_estimators)

fig = px.line(lines, x=lines.index, y=lines.columns,title='Dependence of F-score & GINI score on number of estimators',color_discrete_sequence = colors, labels={
                     "index": "Number of estimators",
                     "value": "Error Value"})
fig.update_traces(line=dict(width=3))
fig.update_layout(xaxis_range=[min(n_estimators)-10,max(n_estimators)+10])

fig.show()

In [60]:
best_n_estimators = 200

### XGBoost with chosen hyperparameters

In [61]:
xgboost = XGBClassifier(random_state=0, n_estimators = best_n_estimators, max_depth = best_max_depth)
xgboost.fit(X_train, y_train)
train_pred = xgboost.predict(X_train)
test_pred = xgboost.predict(X_test)

In [62]:
fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

print('ROC_AUC train:', "%.2f" % roc_auc_score(y_train, train_pred))
print('ROC_AUC test:', "%.2f" % roc_auc_score(y_test, test_pred))

print('F1 train:', "%.2f" % f1_score(y_train, train_pred))
print('F1 test:', "%.2f" % f1_score(y_test, test_pred))

print('GINI train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
print('GINI test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))

ROC_AUC train: 0.96
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.74
GINI train: 0.91
GINI test: 0.41


In [63]:
colors = ["#016450", "#02818a", "#3690c0", "#67a9cf", "#a6bddb", "#d0d1e6", "#ece2f0", "#fff7fb"]
from sklearn.inspection import permutation_importance
sort = xgboost.feature_importances_.argsort()

fig = px.bar(x=xgboost.feature_importances_[sort], y=data.columns[sort],
             orientation='h', color_discrete_sequence=colors, labels={
                     "y": " "
                   })

fig.update_layout(xaxis_title="Feature Importance")  # Update the x-axis label

fig.show()

# Gradient boosting

In [65]:
from sklearn.ensemble import GradientBoostingClassifier

In [66]:
grad_boost = GradientBoostingClassifier(random_state=0)
grad_boost.fit(X_train, y_train)
train_pred = grad_boost.predict(X_train)
test_pred = grad_boost.predict(X_test)

In [67]:
fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

print('ROC_AUC Train:', "%.2f" % roc_auc_score(y_train, train_pred))
print('ROC_AUC Test:', "%.2f" % roc_auc_score(y_test, test_pred))

print('F1 Train:', "%.2f" % f1_score(y_train, train_pred))
print('F1 Test:', "%.2f" % f1_score(y_test, test_pred))

print('GINI Train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
print('GINI Test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))

ROC_AUC Train: 0.61
ROC_AUC Test: 0.61
F1 Train: 0.62
F1 Test: 0.61
GINI Train: 0.23
GINI Test: 0.21


### Selectng hyperparameters

In [68]:
max_depth = [10, 20, 50, 100]

In [69]:
F1_train = []
F1_test = []

GINI_train = []
GINI_test = []
ROC_AUC_train = []
ROC_AUC_test = []

for depth in max_depth:

    grad_boost = GradientBoostingClassifier(random_state=0,max_depth = depth)
    grad_boost.fit(X_train, y_train)
    train_pred = grad_boost.predict(X_train)
    test_pred = grad_boost.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    print('Max depth:', depth)
    print('ROC_AUC train:', "%.2f" % roc_auc_score(y_train, train_pred))
    print('ROC_AUC test:', "%.2f" % roc_auc_score(y_test, test_pred))

    print('F1 train:', "%.2f" % f1_score(y_train, train_pred))
    print('F1 test:', "%.2f" % f1_score(y_test, test_pred))

    print('GINI train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
    print('GINI test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))
    print()
    
    F1_train.append(f1_score(y_train, train_pred))
    GINI_train.append((2 * auc(fpr_train, tpr_train) - 1))
    ROC_AUC_train.append(roc_auc_score(y_train, train_pred))
    
    
    F1_test.append(f1_score(y_test, test_pred))
    GINI_test.append((2 * auc(fpr_test, tpr_test) - 1))
    ROC_AUC_test.append(roc_auc_score(y_test, test_pred))
    

Max depth: 10
ROC_AUC train: 0.81
ROC_AUC test: 0.67
F1 train: 0.81
F1 test: 0.66
GINI train: 0.63
GINI test: 0.33

Max depth: 20
ROC_AUC train: 0.96
ROC_AUC test: 0.70
F1 train: 0.96
F1 test: 0.70
GINI train: 0.91
GINI test: 0.39

Max depth: 50
ROC_AUC train: 0.96
ROC_AUC test: 0.68
F1 train: 0.96
F1 test: 0.69
GINI train: 0.91
GINI test: 0.36

Max depth: 100
ROC_AUC train: 0.96
ROC_AUC test: 0.68
F1 train: 0.96
F1 test: 0.69
GINI train: 0.91
GINI test: 0.36



In [70]:
lines = pd.DataFrame({
"F1 Train" : F1_train,
"F1 Test" : F1_test,
"GINI Train" : GINI_train,
"GINI Test" : GINI_test,
"ROC_AUC Train" : ROC_AUC_train,
"ROC_AUC Test" : ROC_AUC_test,
}, index=max_depth)

fig = px.line(lines, x=lines.index, y=lines.columns,title='Dependence of F-score & GINI score on max_depth',color_discrete_sequence = colors, labels={
                     "index": "Max depth",
                     "value": "Error Value"})
fig.update_traces(line=dict(width=3))
fig.update_layout(xaxis_range=[min(max_depth)-10,max(max_depth)+10])

fig.show()

In [68]:
best_max_depth=20

In [69]:
n_estimators = [50, 70, 150, 200]

In [73]:
F1_train = []
F1_test = []

GINI_train = []
GINI_test = []
ROC_AUC_train = []
ROC_AUC_test = []

for estimators in n_estimators:

    grad_boost = GradientBoostingClassifier(random_state=0, max_depth = best_max_depth, n_estimators = estimators)
    grad_boost.fit(X_train, y_train)
    train_pred = grad_boost.predict(X_train)
    test_pred = grad_boost.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    print('Estimators:', estimators)
    print('ROC_AUC train:', "%.2f" % roc_auc_score(y_train, train_pred))
    print('ROC_AUC test:', "%.2f" % roc_auc_score(y_test, test_pred))

    print('F1 train:', "%.2f" % f1_score(y_train, train_pred))
    print('F1 test:', "%.2f" % f1_score(y_test, test_pred))

    print('GINI train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
    print('GINI test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))
    print()
    
    F1_train.append(f1_score(y_train, train_pred))
    GINI_train.append((2 * auc(fpr_train, tpr_train) - 1))
    ROC_AUC_train.append(roc_auc_score(y_train, train_pred))
    
    
    F1_test.append(f1_score(y_test, test_pred))
    GINI_test.append((2 * auc(fpr_test, tpr_test) - 1))
    ROC_AUC_test.append(roc_auc_score(y_test, test_pred))
    

Estimators: 50
ROC_AUC train: 0.95
ROC_AUC test: 0.69
F1 train: 0.95
F1 test: 0.70
GINI train: 0.90
GINI test: 0.38

Estimators: 70
ROC_AUC train: 0.95
ROC_AUC test: 0.69
F1 train: 0.95
F1 test: 0.70
GINI train: 0.91
GINI test: 0.39

Estimators: 150
ROC_AUC train: 0.96
ROC_AUC test: 0.70
F1 train: 0.96
F1 test: 0.71
GINI train: 0.91
GINI test: 0.40

Estimators: 200
ROC_AUC train: 0.96
ROC_AUC test: 0.70
F1 train: 0.96
F1 test: 0.71
GINI train: 0.92
GINI test: 0.39



In [74]:
lines = pd.DataFrame({
"F1 Train" : F1_train,
"F1 Test" : F1_test,
"GINI Train" : GINI_train,
"GINI Test" : GINI_test,
"ROC_AUC Train" : ROC_AUC_train,
"ROC_AUC Test" : ROC_AUC_test,
}, index=n_estimators)

fig = px.line(lines, x=lines.index, y=lines.columns,title='Dependence of F-score & GINI score on number of estimators',color_discrete_sequence = colors, labels={
                     "index": "Min_samples_split",
                     "value": "Error Value"})
fig.update_traces(line=dict(width=3))
fig.update_layout(xaxis_range=[min(n_estimators)-10,max(n_estimators)+10])

fig.show()

In [70]:
best_n_estimators = 150

### Gradient Boosting with chosen hyperparameters

In [71]:
grad_boost = GradientBoostingClassifier(random_state=0, n_estimators = best_n_estimators, max_depth = best_max_depth)
grad_boost.fit(X_train, y_train)
train_pred = grad_boost.predict(X_train)
test_pred = grad_boost.predict(X_test)

In [72]:
fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

print('ROC_AUC train:', "%.2f" % roc_auc_score(y_train, train_pred))
print('ROC_AUC test:', "%.2f" % roc_auc_score(y_test, test_pred))

print('F1 train:', "%.2f" % f1_score(y_train, train_pred))
print('F1 test:', "%.2f" % f1_score(y_test, test_pred))

print('GINI train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
print('GINI test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))

ROC_AUC train: 0.96
ROC_AUC test: 0.71
F1 train: 0.96
F1 test: 0.74
GINI train: 0.91
GINI test: 0.41


In [74]:
colors = ["#016450", "#02818a", "#3690c0", "#67a9cf", "#a6bddb", "#d0d1e6", "#ece2f0", "#fff7fb"]
from sklearn.inspection import permutation_importance
sort = grad_boost.feature_importances_.argsort()

fig = px.bar(x=grad_boost.feature_importances_[sort], y=data.columns[sort],
             orientation='h', color_discrete_sequence=colors, labels={
                     "y": " "
                   })

fig.update_layout(xaxis_title="Feature Importance")  # Update the x-axis label

fig.show()

# CatBoost

In [75]:
from catboost import CatBoostClassifier

In [76]:
cat_boost = CatBoostClassifier(verbose=0)
cat_boost.fit(X_train, y_train)
train_pred = cat_boost.predict(X_train)
test_pred = cat_boost.predict(X_test)

In [77]:
fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

print('ROC_AUC Train:', "%.2f" % roc_auc_score(y_train, train_pred))
print('ROC_AUC Test:', "%.2f" % roc_auc_score(y_test, test_pred))

print('F1 Train:', "%.2f" % f1_score(y_train, train_pred))
print('F1 Test:', "%.2f" % f1_score(y_test, test_pred))

print('GINI Train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
print('GINI Test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))

ROC_AUC Train: 0.69
ROC_AUC Test: 0.64
F1 Train: 0.72
F1 Test: 0.67
GINI Train: 0.38
GINI Test: 0.27


In [82]:
n_estimators = [20, 50, 100, 150, 200]

In [83]:
F1_train = []
F1_test = []
GINI_train = []
GINI_test = []
ROC_AUC_train = []
ROC_AUC_test = []

for estimators in n_estimators:

    cat_boost = CatBoostClassifier(verbose=0, n_estimators = estimators)
    cat_boost.fit(X_train, y_train)
    train_pred = cat_boost.predict(X_train)
    test_pred = cat_boost.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    print('Estimators:', estimators)
    print('ROC_AUC train:', "%.2f" % roc_auc_score(y_train, train_pred))
    print('ROC_AUC test:', "%.2f" % roc_auc_score(y_test, test_pred))

    print('F1 train:', "%.2f" % f1_score(y_train, train_pred))
    print('F1 test:', "%.2f" % f1_score(y_test, test_pred))

    print('GINI train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
    print('GINI test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))
    print()
    
    F1_train.append(f1_score(y_train, train_pred))
    GINI_train.append((2 * auc(fpr_train, tpr_train) - 1))
    ROC_AUC_train.append(roc_auc_score(y_train, train_pred))
    
    
    F1_test.append(f1_score(y_test, test_pred))
    GINI_test.append((2 * auc(fpr_test, tpr_test) - 1))
    ROC_AUC_test.append(roc_auc_score(y_test, test_pred))
    

Estimators: 20
ROC_AUC train: 0.61
ROC_AUC test: 0.60
F1 train: 0.57
F1 test: 0.56
GINI train: 0.22
GINI test: 0.21

Estimators: 50
ROC_AUC train: 0.63
ROC_AUC test: 0.61
F1 train: 0.61
F1 test: 0.59
GINI train: 0.26
GINI test: 0.23

Estimators: 100
ROC_AUC train: 0.65
ROC_AUC test: 0.62
F1 train: 0.63
F1 test: 0.60
GINI train: 0.29
GINI test: 0.24

Estimators: 150
ROC_AUC train: 0.65
ROC_AUC test: 0.62
F1 train: 0.63
F1 test: 0.60
GINI train: 0.30
GINI test: 0.25

Estimators: 200
ROC_AUC train: 0.67
ROC_AUC test: 0.63
F1 train: 0.65
F1 test: 0.61
GINI train: 0.33
GINI test: 0.26



In [84]:
lines = pd.DataFrame({
"F1 Train" : F1_train,
"F1 Test" : F1_test,
"GINI Train" : GINI_train,
"GINI Test" : GINI_test,
"ROC_AUC Train" : ROC_AUC_train,
"ROC_AUC Test" : ROC_AUC_test,
}, index=n_estimators)

fig = px.line(lines, x=lines.index, y=lines.columns,title='Dependence of F-score & GINI score on number of estimators',color_discrete_sequence = colors, labels={
                     "index": "Number of estimators",
                     "value": "Error Value"})
fig.update_traces(line=dict(width=3))
fig.update_layout(xaxis_range=[min(n_estimators)-10,max(n_estimators)+10])

fig.show()

In [78]:
best_n_estimators = 200

### CatBoost with chosen hyperparameters

In [81]:
cat_boost = CatBoostClassifier(verbose=0,n_estimators = best_n_estimators )
cat_boost.fit(X_train, y_train)
train_pred = cat_boost.predict(X_train)
test_pred = cat_boost.predict(X_test)

In [82]:
fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

print('ROC_AUC Train:', "%.2f" % roc_auc_score(y_train, train_pred))
print('ROC_AUC Test:', "%.2f" % roc_auc_score(y_test, test_pred))

print('F1 Train:', "%.2f" % f1_score(y_train, train_pred))
print('F1 Test:', "%.2f" % f1_score(y_test, test_pred))

print('GINI Train:', "%.2f" % (2 * auc(fpr_train, tpr_train) - 1))
print('GINI Test:', "%.2f" % (2 * auc(fpr_test, tpr_test) - 1))

ROC_AUC Train: 0.67
ROC_AUC Test: 0.63
F1 Train: 0.71
F1 Test: 0.67
GINI Train: 0.35
GINI Test: 0.26


In [83]:
colors = ["#016450", "#02818a", "#3690c0", "#67a9cf", "#a6bddb", "#d0d1e6", "#ece2f0", "#fff7fb"]
from sklearn.inspection import permutation_importance
sort = cat_boost.feature_importances_.argsort()

fig = px.bar(x=cat_boost.feature_importances_[sort], y=data.columns[sort],
             orientation='h', color_discrete_sequence=colors, labels={
                     "y": " "
                   })

fig.update_layout(xaxis_title="Feature Importance")  # Update the x-axis label

fig.show()

# By clusters

In [169]:
data = pd.read_csv('df_12_clusters_km.csv', sep=',')

In [170]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79389 entries, 0 to 79388
Data columns (total 46 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Телефон_new                            79389 non-null  object 
 1   Выручка_bought                         79389 non-null  int64  
 2   Маржа_bought                           79389 non-null  float64
 3   Средний чек_bought                     79389 non-null  int64  
 4   Кол-во чеков_bought                    79389 non-null  int64  
 5   Среднее кол-во товаров в  чеке_bought  79389 non-null  int64  
 6   Центральный ф.о.                       79389 non-null  float64
 7   Приволжский ф.о.                       79389 non-null  float64
 8   Северо-Западный ф.о.                   79389 non-null  float64
 9   Южный ф.о.                             79389 non-null  float64
 10  Сибирский ф.о.                         79389 non-null  float64
 11  Ур

In [171]:
def multiple_models(df,N):
  list_of_results=[]

  for i in range(N):

    results = pd.DataFrame({"RandomForestClassifier":[0]*6,
                        "DecisionTreeClassifier":[0]*6,
                        "XGBoost":[0]*6,
                        "Gradient Boosting":[0]*6,
                        "CatBoost":[0]*6,
                        "Support Vector Machine":[0]*6,
  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])

    X = df[df['Labels']==i].drop(['target'],axis=1)
    y = df[df['Labels']==i]['target']

    # normalization process
    numeric_data = X.select_dtypes([np.number]).copy()
    scaler = StandardScaler()
    numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
    X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

    final_iv, IV = data_vars(X_train.copy(), y_train.copy())

    important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])
    print(f'For cluster {i} there are {len(important_columns)} important columns:{important_columns}\n\n')

    X_train = X_train[important_columns]
    X_test = X_test[important_columns]
    X_valid = X_valid[important_columns]




    # RandomForestClassifier

    F1_train = []
    F1_test = []

    GINI_train = []
    GINI_test = []

    depth=50
    best_n_estimators = 200

    clf_forest = RandomForestClassifier(random_state=609, n_estimators = best_n_estimators, max_depth = depth)
    clf_forest.fit(X_train, y_train)
    train_pred = clf_forest.predict(X_train)
    test_pred = clf_forest.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    
    results['RandomForestClassifier']= [roc_auc_score(y_train, train_pred),roc_auc_score(y_test, test_pred), 
    f1_score(y_train, train_pred), f1_score(y_test, test_pred),
    (2 * auc(fpr_train, tpr_train) - 1), (2 * auc(fpr_test, tpr_test) - 1)]

    #list_of_results.append(results)


    # DecisionTreeClassifier
    depth = 50
    clf_tree = DecisionTreeClassifier(random_state=609, max_depth = depth)
    clf_tree.fit(X_train, y_train)
    train_pred = clf_tree.predict(X_train)
    test_pred = clf_tree.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    results['DecisionTreeClassifier']= [roc_auc_score(y_train, train_pred),roc_auc_score(y_test, test_pred), 
    f1_score(y_train, train_pred), f1_score(y_test, test_pred),
    (2 * auc(fpr_train, tpr_train) - 1), (2 * auc(fpr_test, tpr_test) - 1)]

    #list_of_results.append(results)

    # Support Vector Machine

    from sklearn.svm import SVC
    SVC = SVC(random_state=609)
    SVC.fit(X_train, y_train)
      
    train_pred = SVC.predict(X_train)
    test_pred = SVC.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    results['Support Vector Machine']= [roc_auc_score(y_train, train_pred),roc_auc_score(y_test, test_pred), 
    f1_score(y_train, train_pred), f1_score(y_test, test_pred),
    (2 * auc(fpr_train, tpr_train) - 1), (2 * auc(fpr_test, tpr_test) - 1)]

    #list_of_results.append(results)

    # XGBoost

    xgboost = XGBClassifier()
    xgboost.fit(X_train, y_train)
      
    train_pred = xgboost.predict(X_train)
    test_pred = xgboost.predict(X_test)

    xgboost = XGBClassifier(random_state=0, max_depth = depth)
    xgboost.fit(X_train, y_train)
    train_pred = xgboost.predict(X_train)
    test_pred = xgboost.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    results['XGBoost']= [roc_auc_score(y_train, train_pred),roc_auc_score(y_test, test_pred), 
    f1_score(y_train, train_pred), f1_score(y_test, test_pred),
    (2 * auc(fpr_train, tpr_train) - 1), (2 * auc(fpr_test, tpr_test) - 1)]

    #list_of_results.append(results)

    # Gradient Boosting
    depth = 50
    grad_boost = GradientBoostingClassifier(random_state=609,max_depth = depth)
    grad_boost.fit(X_train, y_train)
    train_pred = grad_boost.predict(X_train)
    test_pred = grad_boost.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    results['Gradient Boosting']= [roc_auc_score(y_train, train_pred),roc_auc_score(y_test, test_pred), 
    f1_score(y_train, train_pred), f1_score(y_test, test_pred),
    (2 * auc(fpr_train, tpr_train) - 1), (2 * auc(fpr_test, tpr_test) - 1)]

    #list_of_results.append(results)



    # CatBoost
    estimators=200
    cat_boost = CatBoostClassifier(random_state=609, verbose=0, n_estimators = estimators)
    cat_boost.fit(X_train, y_train)
    train_pred = cat_boost.predict(X_train)
    test_pred = cat_boost.predict(X_test)

    fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
    fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

    results['CatBoost']= [roc_auc_score(y_train, train_pred),roc_auc_score(y_test, test_pred), 
    f1_score(y_train, train_pred), f1_score(y_test, test_pred),
    (2 * auc(fpr_train, tpr_train) - 1), (2 * auc(fpr_test, tpr_test) - 1)]

    list_of_results.append(results)

  return list_of_results

In [137]:
res = multiple_models(data,12)

For cluster 0 there are 3 important columns:['Выручка_bought' 'Кол-во товаров_bought' 'Кол-во чеков_bought']


For cluster 1 there are 7 important columns:['Абсолютная маржа_bought' 'Выручка_bought' 'КОСМЕТИКА/ГИГИЕНА выручка'
 'Кол-во товаров_bought' 'Кол-во чеков_bought' 'ППКП выручка'
 'Ценность клиента_bought']


For cluster 2 there are 1 important columns:['Средняя маржа_bought']


For cluster 3 there are 14 important columns:['Абсолютная маржа_bought' 'Выручка_bought' 'ДЕТСКОЕ ПИТАНИЕ выручка'
 'ИГРУШКИ выручка' 'ИГРУШКИт выручка' 'КОСМЕТИКА/ГИГИЕНА выручка'
 'Кол-во чеков_bought' 'Маржа_bought' 'ПОДГУЗНИКИ выручка' 'ППКП'
 'ППКП выручка' 'Среднее кол-во товаров в  чеке_bought'
 'Средний чек_bought' 'Ценность клиента_bought']


For cluster 4 there are 13 important columns:['Выручка_bought' 'ИГРУШКИ выручка' 'ИГРУШКИт выручка' 'КГТ выручка'
 'КРУПНОГАБАРИТНЫЙ ТОВАР выручка' 'Кол-во товаров_bought'
 'Кол-во чеков_bought' 'Маржа_bought' 'ОБУВЬ выручка' 'ОДЕЖДА выручка'
 'ППКП выручк

In [138]:
len(res)

12

In [139]:
res[0]

,RandomForestClassifier,DecisionTreeClassifier,XGBoost,Gradient Boosting,CatBoost,Support Vector Machine
ROC-AUC Train,0.929897,0.925583,0.924747,0.929823,0.652786,0.547617
ROC-AUC Test,0.618321,0.611908,0.625439,0.614850,0.582501,0.535506
F1 Train,0.929993,0.923194,0.924500,0.931358,0.612664,0.432007
F1 Test,0.628128,0.599360,0.631854,0.618086,0.538612,0.427248
GINI Train,0.859793,0.851166,0.849493,0.859646,0.305572,0.095234
GINI Test,0.236642,0.223817,0.250878,0.229700,0.165002,0.071012


In [140]:
res[1]

,RandomForestClassifier,DecisionTreeClassifier,XGBoost,Gradient Boosting,CatBoost,Support Vector Machine
ROC-AUC Train,0.978580,0.978653,0.978515,0.978511,0.709846,0.571079
ROC-AUC Test,0.695030,0.676376,0.688156,0.675645,0.630338,0.567222
F1 Train,0.978771,0.978602,0.978672,0.978929,0.707144,0.521411
F1 Test,0.705605,0.678533,0.695867,0.681256,0.631617,0.517661
GINI Train,0.957160,0.957306,0.957030,0.957023,0.419691,0.142159
GINI Test,0.390060,0.352752,0.376311,0.351289,0.260676,0.134445


In [141]:
res[2]

,RandomForestClassifier,DecisionTreeClassifier,XGBoost,Gradient Boosting,CatBoost,Support Vector Machine
ROC-AUC Train,0.805373,0.802299,0.777899,0.802299,0.592095,0.514744
ROC-AUC Test,0.643091,0.641545,0.636454,0.641545,0.557292,0.519054
F1 Train,0.790596,0.779459,0.760680,0.779459,0.519198,0.219048
F1 Test,0.612180,0.593620,0.604332,0.593620,0.468223,0.218647
GINI Train,0.610746,0.604598,0.555797,0.604598,0.184191,0.029487
GINI Test,0.286181,0.283091,0.272908,0.283091,0.114583,0.038108


In [142]:
res[3]

,RandomForestClassifier,DecisionTreeClassifier,XGBoost,Gradient Boosting,CatBoost,Support Vector Machine
ROC-AUC Train,0.996765,0.997854,0.997037,0.996220,0.785385,0.501532
ROC-AUC Test,0.649206,0.648204,0.664681,0.647146,0.619222,0.500000
F1 Train,0.997854,0.997849,0.997853,0.997856,0.896460,0.811219
F1 Test,0.820947,0.769453,0.811816,0.778252,0.810034,0.810761
GINI Train,0.993530,0.995708,0.994074,0.992441,0.570770,0.003064
GINI Test,0.298411,0.296409,0.329362,0.294292,0.238443,0.000000


In [143]:
res[4]

,RandomForestClassifier,DecisionTreeClassifier,XGBoost,Gradient Boosting,CatBoost,Support Vector Machine
ROC-AUC Train,0.973578,0.974015,0.973665,0.973132,0.705919,0.547625
ROC-AUC Test,0.639310,0.626503,0.634655,0.629677,0.594174,0.541594
F1 Train,0.974425,0.974092,0.974359,0.974756,0.706411,0.637277
F1 Test,0.656535,0.635813,0.650702,0.646351,0.591491,0.630358
GINI Train,0.947156,0.948030,0.947331,0.946264,0.411838,0.095249
GINI Test,0.278620,0.253006,0.269310,0.259354,0.188349,0.083188


In [144]:
res[5]

,RandomForestClassifier,DecisionTreeClassifier,XGBoost,Gradient Boosting,CatBoost,Support Vector Machine
ROC-AUC Train,0.867737,0.866427,0.859278,0.866427,0.651032,0.531764
ROC-AUC Test,0.622916,0.628221,0.631943,0.624045,0.558031,0.522070
F1 Train,0.865965,0.856499,0.856537,0.856499,0.651452,0.480309
F1 Test,0.608470,0.587581,0.615652,0.582560,0.558140,0.454198
GINI Train,0.735473,0.732854,0.718556,0.732854,0.302065,0.063528
GINI Test,0.245832,0.256441,0.263885,0.248091,0.116061,0.044140


In [145]:
res[6]

,RandomForestClassifier,DecisionTreeClassifier,XGBoost,Gradient Boosting,CatBoost,Support Vector Machine
ROC-AUC Train,1.000000,1.000000,1.000000,1.000000,1.000000,0.690476
ROC-AUC Test,0.642361,0.614583,0.642361,0.614583,0.704861,0.607639
F1 Train,1.000000,1.000000,1.000000,1.000000,1.000000,0.745098
F1 Test,0.600000,0.580645,0.600000,0.580645,0.687500,0.681818
GINI Train,1.000000,1.000000,1.000000,1.000000,1.000000,0.380952
GINI Test,0.284722,0.229167,0.284722,0.229167,0.409722,0.215278


In [146]:
res[7]

,RandomForestClassifier,DecisionTreeClassifier,XGBoost,Gradient Boosting,CatBoost,Support Vector Machine
ROC-AUC Train,0.989184,0.988962,0.988468,0.988962,0.718820,0.571915
ROC-AUC Test,0.638553,0.628719,0.645745,0.635197,0.596008,0.545468
F1 Train,0.988366,0.988331,0.987393,0.988331,0.653823,0.399449
F1 Test,0.597730,0.590654,0.607769,0.596623,0.501071,0.350168
GINI Train,0.978367,0.977925,0.976935,0.977925,0.437640,0.143830
GINI Test,0.277106,0.257437,0.291491,0.270395,0.192016,0.090937


In [147]:
res[8]

,RandomForestClassifier,DecisionTreeClassifier,XGBoost,Gradient Boosting,CatBoost,Support Vector Machine
ROC-AUC Train,1.000000,1.000000,1.000000,1.000000,1.000000,0.5
ROC-AUC Test,0.678571,0.535714,0.583333,0.571429,0.678571,0.5
F1 Train,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
F1 Test,0.545455,0.400000,0.285714,0.428571,0.545455,0.0
GINI Train,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
GINI Test,0.357143,0.071429,0.166667,0.142857,0.357143,0.0


In [148]:
res[9]

,RandomForestClassifier,DecisionTreeClassifier,XGBoost,Gradient Boosting,CatBoost,Support Vector Machine
ROC-AUC Train,0.946437,0.946632,0.939574,0.946205,0.662684,0.567664
ROC-AUC Test,0.679054,0.674105,0.674071,0.671218,0.613366,0.561918
F1 Train,0.946237,0.945301,0.939445,0.947307,0.614956,0.382356
F1 Test,0.675418,0.669681,0.673708,0.677101,0.561870,0.368543
GINI Train,0.892874,0.893264,0.879147,0.892411,0.325369,0.135328
GINI Test,0.358107,0.348210,0.348141,0.342436,0.226731,0.123836


In [149]:
res[10]

,RandomForestClassifier,DecisionTreeClassifier,XGBoost,Gradient Boosting,CatBoost,Support Vector Machine
ROC-AUC Train,1.000000,1.000000,1.000000,1.000000,0.919312,0.636243
ROC-AUC Test,0.674633,0.620755,0.709644,0.628512,0.666247,0.641719
F1 Train,1.000000,1.000000,1.000000,1.000000,0.912621,0.487179
F1 Test,0.659574,0.593407,0.707071,0.590909,0.619048,0.507463
GINI Train,1.000000,1.000000,1.000000,1.000000,0.838624,0.272487
GINI Test,0.349266,0.241509,0.419287,0.257023,0.332495,0.283438


In [150]:
res[11]

,RandomForestClassifier,DecisionTreeClassifier,XGBoost,Gradient Boosting,CatBoost,Support Vector Machine
ROC-AUC Train,0.971566,0.975564,0.973165,0.970766,0.764607,0.502618
ROC-AUC Test,0.649604,0.644078,0.642454,0.613128,0.602688,0.500000
F1 Train,0.976321,0.975881,0.976147,0.976407,0.828767,0.743243
F1 Test,0.728421,0.683486,0.709957,0.674058,0.721893,0.741100
GINI Train,0.943132,0.951128,0.946330,0.941533,0.529215,0.005236
GINI Test,0.299209,0.288155,0.284907,0.226255,0.205377,0.000000


## Response prediction

### Cluster 0

In [351]:
data = pd.read_csv('df_12_clusters_km.csv', sep=',')

In [352]:
cluster_0 = data[data['Labels'] == 0]

In [353]:
cluster_0

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
29,55575657-55485751525479,12464,0.24,4155,3,3,0.0,0.0,0.0,1.0,...,0,2992,1,997.33,2992,3,12464.0,0.0,0.0,0.0
31,55574857-51574948505274,16434,0.39,3689,5,2,0.0,0.0,0.0,1.0,...,0,6359,1,1271.80,6359,5,13147.2,0.0,3286.8,0.0
40,55574857-52525255485279,3040,0.43,3040,1,1,0.0,0.0,0.0,1.0,...,0,1314,1,1314.00,1314,1,3040.0,0.0,0.0,0.0
41,55574852-52525753535670,2692,0.68,538,3,5,0.0,0.0,0.0,1.0,...,0,1842,1,614.00,1842,8,0.0,0.0,0.0,2692.0
50,55575056-52485553515670,1358,0.39,340,4,4,0.0,0.0,0.0,1.0,...,0,531,1,132.75,531,4,0.0,0.0,1358.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79318,55575655-56535554535677,2049,0.40,2049,1,1,0.0,0.0,0.0,1.0,...,1,819,1,819.00,819,1,0.0,0.0,2049.0,0.0
79325,55574856-54495557494975,449,0.28,449,1,1,0.0,0.0,0.0,1.0,...,1,126,1,126.00,126,1,0.0,0.0,0.0,0.0
79340,55574853-52485752525270,2022,0.33,2022,1,1,0.0,0.0,0.0,1.0,...,1,674,1,674.00,674,1,0.0,0.0,2022.0,0.0
79368,55575056-49505252514973,2576,0.36,2576,1,1,0.0,0.0,0.0,1.0,...,1,920,1,920.00,920,1,2576.0,0.0,0.0,0.0


In [354]:
X = data[data['Labels']==0].drop(['target'],axis=1)
y = data[data['Labels']==0]['target']

In [355]:
X

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Labels,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
29,55575657-55485751525479,12464,0.24,4155,3,3,0.0,0.0,0.0,1.0,...,0,2992,1,997.33,2992,3,12464.0,0.0,0.0,0.0
31,55574857-51574948505274,16434,0.39,3689,5,2,0.0,0.0,0.0,1.0,...,0,6359,1,1271.80,6359,5,13147.2,0.0,3286.8,0.0
40,55574857-52525255485279,3040,0.43,3040,1,1,0.0,0.0,0.0,1.0,...,0,1314,1,1314.00,1314,1,3040.0,0.0,0.0,0.0
41,55574852-52525753535670,2692,0.68,538,3,5,0.0,0.0,0.0,1.0,...,0,1842,1,614.00,1842,8,0.0,0.0,0.0,2692.0
50,55575056-52485553515670,1358,0.39,340,4,4,0.0,0.0,0.0,1.0,...,0,531,1,132.75,531,4,0.0,0.0,1358.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79318,55575655-56535554535677,2049,0.40,2049,1,1,0.0,0.0,0.0,1.0,...,0,819,1,819.00,819,1,0.0,0.0,2049.0,0.0
79325,55574856-54495557494975,449,0.28,449,1,1,0.0,0.0,0.0,1.0,...,0,126,1,126.00,126,1,0.0,0.0,0.0,0.0
79340,55574853-52485752525270,2022,0.33,2022,1,1,0.0,0.0,0.0,1.0,...,0,674,1,674.00,674,1,0.0,0.0,2022.0,0.0
79368,55575056-49505252514973,2576,0.36,2576,1,1,0.0,0.0,0.0,1.0,...,0,920,1,920.00,920,1,2576.0,0.0,0.0,0.0


In [356]:
results = pd.DataFrame({
                        "XGBoost":[0]*6,

  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])
# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

final_iv, IV = data_vars(X_train.copy(), y_train.copy())

important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]
X = X[important_columns]    
depth=70

xgboost = XGBClassifier(random_state=0, max_depth = depth)
xgboost.fit(X_train, y_train)
train_pred = xgboost.predict(X_train)
test_pred = xgboost.predict(X_test)

predictions = xgboost.predict_proba(X)
print(predictions)



[[0.06489509 0.9351049 ]
 [0.57159084 0.42840916]
 [0.9454994  0.05450055]
 ...
 [0.23316956 0.76683044]
 [0.23534    0.76466   ]
 [0.18911684 0.81088316]]


In [357]:
cluster_0['predictions'] = 0

In [358]:
cluster_0

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
29,55575657-55485751525479,12464,0.24,4155,3,3,0.0,0.0,0.0,1.0,...,2992,1,997.33,2992,3,12464.0,0.0,0.0,0.0,0
31,55574857-51574948505274,16434,0.39,3689,5,2,0.0,0.0,0.0,1.0,...,6359,1,1271.80,6359,5,13147.2,0.0,3286.8,0.0,0
40,55574857-52525255485279,3040,0.43,3040,1,1,0.0,0.0,0.0,1.0,...,1314,1,1314.00,1314,1,3040.0,0.0,0.0,0.0,0
41,55574852-52525753535670,2692,0.68,538,3,5,0.0,0.0,0.0,1.0,...,1842,1,614.00,1842,8,0.0,0.0,0.0,2692.0,0
50,55575056-52485553515670,1358,0.39,340,4,4,0.0,0.0,0.0,1.0,...,531,1,132.75,531,4,0.0,0.0,1358.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79318,55575655-56535554535677,2049,0.40,2049,1,1,0.0,0.0,0.0,1.0,...,819,1,819.00,819,1,0.0,0.0,2049.0,0.0,0
79325,55574856-54495557494975,449,0.28,449,1,1,0.0,0.0,0.0,1.0,...,126,1,126.00,126,1,0.0,0.0,0.0,0.0,0
79340,55574853-52485752525270,2022,0.33,2022,1,1,0.0,0.0,0.0,1.0,...,674,1,674.00,674,1,0.0,0.0,2022.0,0.0,0
79368,55575056-49505252514973,2576,0.36,2576,1,1,0.0,0.0,0.0,1.0,...,920,1,920.00,920,1,2576.0,0.0,0.0,0.0,0


In [359]:
cluster_0['predictions']=predictions

In [360]:
cluster_0

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
29,55575657-55485751525479,12464,0.24,4155,3,3,0.0,0.0,0.0,1.0,...,2992,1,997.33,2992,3,12464.0,0.0,0.0,0.0,0.064895
31,55574857-51574948505274,16434,0.39,3689,5,2,0.0,0.0,0.0,1.0,...,6359,1,1271.80,6359,5,13147.2,0.0,3286.8,0.0,0.571591
40,55574857-52525255485279,3040,0.43,3040,1,1,0.0,0.0,0.0,1.0,...,1314,1,1314.00,1314,1,3040.0,0.0,0.0,0.0,0.945499
41,55574852-52525753535670,2692,0.68,538,3,5,0.0,0.0,0.0,1.0,...,1842,1,614.00,1842,8,0.0,0.0,0.0,2692.0,0.979691
50,55575056-52485553515670,1358,0.39,340,4,4,0.0,0.0,0.0,1.0,...,531,1,132.75,531,4,0.0,0.0,1358.0,0.0,0.912032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79318,55575655-56535554535677,2049,0.40,2049,1,1,0.0,0.0,0.0,1.0,...,819,1,819.00,819,1,0.0,0.0,2049.0,0.0,0.249749
79325,55574856-54495557494975,449,0.28,449,1,1,0.0,0.0,0.0,1.0,...,126,1,126.00,126,1,0.0,0.0,0.0,0.0,0.092008
79340,55574853-52485752525270,2022,0.33,2022,1,1,0.0,0.0,0.0,1.0,...,674,1,674.00,674,1,0.0,0.0,2022.0,0.0,0.233170
79368,55575056-49505252514973,2576,0.36,2576,1,1,0.0,0.0,0.0,1.0,...,920,1,920.00,920,1,2576.0,0.0,0.0,0.0,0.235340


### Cluster 1

In [361]:
cluster_1 = data[data['Labels'] == 1]

In [362]:
cluster_1

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
22,55575049-51535556505075,1896,0.17,474,4,4,0.0,0.0,1.0,0.0,...,0,327,1,81.75,327,5,474.0,0.0,0.0,474.0
39,55574848-53564953564870,4154,0.08,1791,4,2,1.0,0.0,0.0,0.0,...,1,346,1,86.50,346,10,0.0,0.0,0.0,1038.5
56,55575653-52574856515174,2253,0.15,751,3,3,1.0,0.0,0.0,0.0,...,0,345,1,115.00,345,3,0.0,0.0,0.0,0.0
75,55575054-53534956494971,3245,0.43,270,9,12,1.0,0.0,0.0,0.0,...,0,1402,1,155.78,1402,12,0.0,0.0,0.0,1362.9
81,55574851-48535553505372,2218,0.22,444,5,5,0.0,1.0,0.0,0.0,...,0,488,1,97.60,488,5,0.0,0.0,0.0,887.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79376,55574851-54555254494972,1344,0.15,672,2,2,1.0,0.0,0.0,0.0,...,1,203,1,101.50,203,4,0.0,0.0,0.0,0.0
79378,55575349-53545653495671,1338,0.14,1338,1,1,1.0,0.0,0.0,0.0,...,1,193,1,193.00,193,1,0.0,0.0,0.0,0.0
79381,55575349-51495752515378,766,0.02,766,1,1,1.0,0.0,0.0,0.0,...,1,16,1,16.00,16,1,0.0,0.0,0.0,0.0
79385,55574854-55505648555577,1616,0.05,1616,1,1,1.0,0.0,0.0,0.0,...,1,75,1,75.00,75,8,0.0,0.0,0.0,0.0


In [363]:
X = data[data['Labels']==1].drop(['target'],axis=1)
y = data[data['Labels']==1]['target']

In [364]:
results = pd.DataFrame({
                        "RandomForestClassifier":[0]*6,

  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])
# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

final_iv, IV = data_vars(X_train.copy(), y_train.copy())

important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]
X = X[important_columns]    
depth=50
best_n_estimators = 200

clf_forest = RandomForestClassifier(random_state=609, n_estimators = best_n_estimators, max_depth = depth)
clf_forest.fit(X_train, y_train)
train_pred = clf_forest.predict(X_train)
test_pred = clf_forest.predict(X_test)

predictions = clf_forest.predict_proba(X)
print(predictions)

[[0.82       0.18      ]
 [0.08       0.92      ]
 [0.25       0.75      ]
 ...
 [0.265      0.735     ]
 [0.70533333 0.29466667]
 [0.10116667 0.89883333]]


In [365]:
cluster_1['predictions'] = 0
cluster_1

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
22,55575049-51535556505075,1896,0.17,474,4,4,0.0,0.0,1.0,0.0,...,327,1,81.75,327,5,474.0,0.0,0.0,474.0,0
39,55574848-53564953564870,4154,0.08,1791,4,2,1.0,0.0,0.0,0.0,...,346,1,86.50,346,10,0.0,0.0,0.0,1038.5,0
56,55575653-52574856515174,2253,0.15,751,3,3,1.0,0.0,0.0,0.0,...,345,1,115.00,345,3,0.0,0.0,0.0,0.0,0
75,55575054-53534956494971,3245,0.43,270,9,12,1.0,0.0,0.0,0.0,...,1402,1,155.78,1402,12,0.0,0.0,0.0,1362.9,0
81,55574851-48535553505372,2218,0.22,444,5,5,0.0,1.0,0.0,0.0,...,488,1,97.60,488,5,0.0,0.0,0.0,887.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79376,55574851-54555254494972,1344,0.15,672,2,2,1.0,0.0,0.0,0.0,...,203,1,101.50,203,4,0.0,0.0,0.0,0.0,0
79378,55575349-53545653495671,1338,0.14,1338,1,1,1.0,0.0,0.0,0.0,...,193,1,193.00,193,1,0.0,0.0,0.0,0.0,0
79381,55575349-51495752515378,766,0.02,766,1,1,1.0,0.0,0.0,0.0,...,16,1,16.00,16,1,0.0,0.0,0.0,0.0,0
79385,55574854-55505648555577,1616,0.05,1616,1,1,1.0,0.0,0.0,0.0,...,75,1,75.00,75,8,0.0,0.0,0.0,0.0,0


In [366]:
cluster_1['predictions']=predictions
cluster_1

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
22,55575049-51535556505075,1896,0.17,474,4,4,0.0,0.0,1.0,0.0,...,327,1,81.75,327,5,474.0,0.0,0.0,474.0,0.820000
39,55574848-53564953564870,4154,0.08,1791,4,2,1.0,0.0,0.0,0.0,...,346,1,86.50,346,10,0.0,0.0,0.0,1038.5,0.080000
56,55575653-52574856515174,2253,0.15,751,3,3,1.0,0.0,0.0,0.0,...,345,1,115.00,345,3,0.0,0.0,0.0,0.0,0.250000
75,55575054-53534956494971,3245,0.43,270,9,12,1.0,0.0,0.0,0.0,...,1402,1,155.78,1402,12,0.0,0.0,0.0,1362.9,0.840000
81,55574851-48535553505372,2218,0.22,444,5,5,0.0,1.0,0.0,0.0,...,488,1,97.60,488,5,0.0,0.0,0.0,887.2,0.830000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79376,55574851-54555254494972,1344,0.15,672,2,2,1.0,0.0,0.0,0.0,...,203,1,101.50,203,4,0.0,0.0,0.0,0.0,0.209167
79378,55575349-53545653495671,1338,0.14,1338,1,1,1.0,0.0,0.0,0.0,...,193,1,193.00,193,1,0.0,0.0,0.0,0.0,0.194435
79381,55575349-51495752515378,766,0.02,766,1,1,1.0,0.0,0.0,0.0,...,16,1,16.00,16,1,0.0,0.0,0.0,0.0,0.265000
79385,55574854-55505648555577,1616,0.05,1616,1,1,1.0,0.0,0.0,0.0,...,75,1,75.00,75,8,0.0,0.0,0.0,0.0,0.705333


### Cluster 2

In [367]:
cluster_2 = data[data['Labels'] == 2]
cluster_2

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
5,55575349-49505555535574,7910,0.37,3955,2,2,0.0,0.0,0.0,0.0,...,0,2891,1,1445.5,2891,2,7910.0,0.0,0.0,0.0
24,55574954-48535255495174,1520,0.51,1520,1,1,1.0,0.0,0.0,0.0,...,0,777,1,777.0,777,1,1520.0,0.0,0.0,0.0
27,55574955-53525351575379,2430,0.25,1215,2,2,1.0,0.0,0.0,0.0,...,0,602,1,301.0,602,2,2430.0,0.0,0.0,0.0
33,55575750-51534851535574,5299,0.59,5299,1,1,0.0,0.0,0.0,0.0,...,0,3150,1,3150.0,3150,1,5299.0,0.0,0.0,0.0
45,55574851-55555251495475,3135,0.36,3135,1,1,1.0,0.0,0.0,0.0,...,0,1126,1,1126.0,1126,1,3135.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79306,55574853-55555050505274,770,0.25,770,1,1,1.0,0.0,0.0,0.0,...,1,194,1,194.0,194,1,770.0,0.0,0.0,0.0
79315,55575450-52555157525073,770,0.25,770,1,1,0.0,1.0,0.0,0.0,...,1,194,1,194.0,194,1,770.0,0.0,0.0,0.0
79316,55575351-55525051504872,770,0.25,770,1,1,1.0,0.0,0.0,0.0,...,1,194,1,194.0,194,1,770.0,0.0,0.0,0.0
79353,55575448-48525351565076,1620,0.29,1620,1,1,0.0,1.0,0.0,0.0,...,1,468,1,468.0,468,2,1620.0,0.0,0.0,0.0


In [368]:
X = data[data['Labels']==2].drop(['target'],axis=1)
y = data[data['Labels']==2]['target']

In [369]:
results = pd.DataFrame({
                        "RandomForestClassifier":[0]*6,

  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])
# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

final_iv, IV = data_vars(X_train.copy(), y_train.copy())

important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]
X = X[important_columns]    
depth=50
best_n_estimators = 200

clf_forest = RandomForestClassifier(random_state=609, n_estimators = best_n_estimators, max_depth = depth)
clf_forest.fit(X_train, y_train)
train_pred = clf_forest.predict(X_train)
test_pred = clf_forest.predict(X_test)

predictions = clf_forest.predict_proba(X)
print(predictions)

[[0.5995     0.4005    ]
 [0.64732095 0.35267905]
 [0.95515329 0.04484671]
 ...
 [0.26784127 0.73215873]
 [0.01       0.99      ]
 [0.02291667 0.97708333]]


In [370]:
cluster_2['predictions'] = 0
cluster_2['predictions']=predictions
cluster_2

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
5,55575349-49505555535574,7910,0.37,3955,2,2,0.0,0.0,0.0,0.0,...,2891,1,1445.5,2891,2,7910.0,0.0,0.0,0.0,0.599500
24,55574954-48535255495174,1520,0.51,1520,1,1,1.0,0.0,0.0,0.0,...,777,1,777.0,777,1,1520.0,0.0,0.0,0.0,0.647321
27,55574955-53525351575379,2430,0.25,1215,2,2,1.0,0.0,0.0,0.0,...,602,1,301.0,602,2,2430.0,0.0,0.0,0.0,0.955153
33,55575750-51534851535574,5299,0.59,5299,1,1,0.0,0.0,0.0,0.0,...,3150,1,3150.0,3150,1,5299.0,0.0,0.0,0.0,0.820000
45,55574851-55555251495475,3135,0.36,3135,1,1,1.0,0.0,0.0,0.0,...,1126,1,1126.0,1126,1,3135.0,0.0,0.0,0.0,0.954275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79306,55574853-55555050505274,770,0.25,770,1,1,1.0,0.0,0.0,0.0,...,194,1,194.0,194,1,770.0,0.0,0.0,0.0,0.267841
79315,55575450-52555157525073,770,0.25,770,1,1,0.0,1.0,0.0,0.0,...,194,1,194.0,194,1,770.0,0.0,0.0,0.0,0.267841
79316,55575351-55525051504872,770,0.25,770,1,1,1.0,0.0,0.0,0.0,...,194,1,194.0,194,1,770.0,0.0,0.0,0.0,0.267841
79353,55575448-48525351565076,1620,0.29,1620,1,1,0.0,1.0,0.0,0.0,...,468,1,468.0,468,2,1620.0,0.0,0.0,0.0,0.010000


### Cluster 3

In [371]:
cluster_3 = data[data['Labels'] == 3]

X = data[data['Labels']==3].drop(['target'],axis=1)
y = data[data['Labels']==3]['target']
cluster_3

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
4,55574857-54544951504973,6381,0.14,456,14,14,1.0,0.0,0.0,0.0,...,0,921,1,65.79,921,17,446.67,0.0,1340.01,893.34
7,55574954-56525649565270,7128,0.03,670,9,6,1.0,0.0,0.0,0.0,...,1,204,1,22.67,204,37,0.00,0.0,1283.04,641.52
17,55574954-49495653485179,13265,-0.00,3158,8,4,1.0,0.0,0.0,0.0,...,1,-54,1,-6.75,-54,94,0.00,0.0,0.00,0.00
20,55574857-57525052494870,7381,0.27,388,18,19,1.0,0.0,0.0,0.0,...,0,2008,1,111.56,2008,20,1550.01,0.0,0.00,369.05
35,55575054-56515755505073,8802,0.17,529,21,8,1.0,0.0,0.0,0.0,...,1,1459,1,69.48,1459,37,352.08,0.0,3520.80,1760.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79242,55575349-57495152485471,12999,0.23,12999,1,1,0.0,1.0,0.0,0.0,...,1,2999,1,2999.00,2999,1,0.00,0.0,0.00,0.00
79324,55574851-50574849535070,7640,-0.01,7640,1,1,1.0,0.0,0.0,0.0,...,1,-71,1,-71.00,-71,10,0.00,0.0,0.00,0.00
79337,55575054-50485548545177,7250,0.07,1208,6,6,1.0,0.0,0.0,0.0,...,1,488,1,81.33,488,6,0.00,0.0,0.00,0.00
79348,55575057-53555152515379,6060,0.05,3030,2,1,1.0,0.0,0.0,0.0,...,1,280,1,140.00,280,30,0.00,0.0,0.00,0.00


In [372]:
results = pd.DataFrame({
                        "XGBoost":[0]*6,

  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])
# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

final_iv, IV = data_vars(X_train.copy(), y_train.copy())

important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]
X = X[important_columns]    
depth=70

xgboost = XGBClassifier(random_state=0, max_depth = depth)
xgboost.fit(X_train, y_train)
train_pred = xgboost.predict(X_train)
test_pred = xgboost.predict(X_test)

predictions = xgboost.predict_proba(X)
print(predictions)

[[2.1011865e-01 7.8988135e-01]
 [4.4625998e-04 9.9955374e-01]
 [4.6388805e-02 9.5361120e-01]
 ...
 [4.2251945e-03 9.9577481e-01]
 [4.8640132e-02 9.5135987e-01]
 [1.2309194e-02 9.8769081e-01]]


In [373]:
cluster_3['predictions'] = 0
cluster_3['predictions']=predictions
cluster_3

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
4,55574857-54544951504973,6381,0.14,456,14,14,1.0,0.0,0.0,0.0,...,921,1,65.79,921,17,446.67,0.0,1340.01,893.34,0.210119
7,55574954-56525649565270,7128,0.03,670,9,6,1.0,0.0,0.0,0.0,...,204,1,22.67,204,37,0.00,0.0,1283.04,641.52,0.000446
17,55574954-49495653485179,13265,-0.00,3158,8,4,1.0,0.0,0.0,0.0,...,-54,1,-6.75,-54,94,0.00,0.0,0.00,0.00,0.046389
20,55574857-57525052494870,7381,0.27,388,18,19,1.0,0.0,0.0,0.0,...,2008,1,111.56,2008,20,1550.01,0.0,0.00,369.05,0.987399
35,55575054-56515755505073,8802,0.17,529,21,8,1.0,0.0,0.0,0.0,...,1459,1,69.48,1459,37,352.08,0.0,3520.80,1760.40,0.002905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79242,55575349-57495152485471,12999,0.23,12999,1,1,0.0,1.0,0.0,0.0,...,2999,1,2999.00,2999,1,0.00,0.0,0.00,0.00,0.500257
79324,55574851-50574849535070,7640,-0.01,7640,1,1,1.0,0.0,0.0,0.0,...,-71,1,-71.00,-71,10,0.00,0.0,0.00,0.00,0.021402
79337,55575054-50485548545177,7250,0.07,1208,6,6,1.0,0.0,0.0,0.0,...,488,1,81.33,488,6,0.00,0.0,0.00,0.00,0.004225
79348,55575057-53555152515379,6060,0.05,3030,2,1,1.0,0.0,0.0,0.0,...,280,1,140.00,280,30,0.00,0.0,0.00,0.00,0.048640


### Cluster 4

In [374]:
cluster_4 = data[data['Labels'] == 4]

X = data[data['Labels']==4].drop(['target'],axis=1)
y = data[data['Labels']==4]['target']
cluster_4

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
1,55574851-50484949535070,1698,0.26,849,2,2,1.0,0.0,0.0,0.0,...,0,444,1,222.0,444,2,0.0,0.0,0.00,1698.00
3,55574954-53555555545178,1280,0.39,427,2,3,1.0,0.0,0.0,0.0,...,0,501,1,250.5,501,3,0.0,0.0,0.00,1280.00
6,55574854-55525053544978,1101,0.42,550,2,2,1.0,0.0,0.0,0.0,...,0,459,1,229.5,459,2,550.5,0.0,0.00,550.50
8,55575351-51495656575170,2622,0.29,437,6,6,1.0,0.0,0.0,0.0,...,0,759,1,126.5,759,6,0.0,0.0,445.74,1756.74
9,55574949-57545255564978,2398,0.30,1199,2,2,0.0,0.0,1.0,0.0,...,0,714,1,357.0,714,2,0.0,0.0,0.00,2398.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79377,55574853-53545249505575,1660,0.17,1660,1,1,1.0,0.0,0.0,0.0,...,1,280,1,280.0,280,1,0.0,0.0,0.00,1660.00
79380,55575057-54485053525778,1030,0.23,1030,1,1,1.0,0.0,0.0,0.0,...,1,240,1,240.0,240,1,0.0,0.0,0.00,1030.00
79382,55574854-48525649505279,1390,0.23,1390,1,1,1.0,0.0,0.0,0.0,...,1,320,1,320.0,320,1,0.0,0.0,0.00,1390.00
79383,55575648-55524855495173,2090,0.17,2090,1,1,1.0,0.0,0.0,0.0,...,1,350,1,350.0,350,1,0.0,0.0,0.00,2090.00


In [375]:
results = pd.DataFrame({
                        "RandomForestClassifier":[0]*6,

  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])
# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

final_iv, IV = data_vars(X_train.copy(), y_train.copy())

important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]
X = X[important_columns]    
depth=50
best_n_estimators = 200

clf_forest = RandomForestClassifier(random_state=609, n_estimators = best_n_estimators, max_depth = depth)
clf_forest.fit(X_train, y_train)
train_pred = clf_forest.predict(X_train)
test_pred = clf_forest.predict(X_test)

predictions = clf_forest.predict_proba(X)
print(predictions)

[[0.68279412 0.31720588]
 [0.335      0.665     ]
 [0.29625    0.70375   ]
 ...
 [0.18091667 0.81908333]
 [0.45441667 0.54558333]
 [0.24       0.76      ]]


In [376]:
cluster_4['predictions'] = 0
cluster_4['predictions']=predictions
cluster_4

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
1,55574851-50484949535070,1698,0.26,849,2,2,1.0,0.0,0.0,0.0,...,444,1,222.0,444,2,0.0,0.0,0.00,1698.00,0.682794
3,55574954-53555555545178,1280,0.39,427,2,3,1.0,0.0,0.0,0.0,...,501,1,250.5,501,3,0.0,0.0,0.00,1280.00,0.335000
6,55574854-55525053544978,1101,0.42,550,2,2,1.0,0.0,0.0,0.0,...,459,1,229.5,459,2,550.5,0.0,0.00,550.50,0.296250
8,55575351-51495656575170,2622,0.29,437,6,6,1.0,0.0,0.0,0.0,...,759,1,126.5,759,6,0.0,0.0,445.74,1756.74,0.830000
9,55574949-57545255564978,2398,0.30,1199,2,2,0.0,0.0,1.0,0.0,...,714,1,357.0,714,2,0.0,0.0,0.00,2398.00,0.738042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79377,55574853-53545249505575,1660,0.17,1660,1,1,1.0,0.0,0.0,0.0,...,280,1,280.0,280,1,0.0,0.0,0.00,1660.00,0.545000
79380,55575057-54485053525778,1030,0.23,1030,1,1,1.0,0.0,0.0,0.0,...,240,1,240.0,240,1,0.0,0.0,0.00,1030.00,0.277988
79382,55574854-48525649505279,1390,0.23,1390,1,1,1.0,0.0,0.0,0.0,...,320,1,320.0,320,1,0.0,0.0,0.00,1390.00,0.180917
79383,55575648-55524855495173,2090,0.17,2090,1,1,1.0,0.0,0.0,0.0,...,350,1,350.0,350,1,0.0,0.0,0.00,2090.00,0.454417


### Cluster 5

In [377]:
cluster_5 = data[data['Labels'] == 5]

X = data[data['Labels']==5].drop(['target'],axis=1)
y = data[data['Labels']==5]['target']
cluster_5

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
37,55575453-57485153485077,11990,0.39,11990,1,1,0.0,0.0,0.0,0.0,...,0,4651,1,4651.00,4651,1,11990.0,0.0,0.0,0.0
43,55575351-57494852515273,12350,0.44,12350,1,1,0.0,0.0,0.0,0.0,...,0,5456,1,5456.00,5456,1,12350.0,0.0,0.0,0.0
44,55574854-53495257575179,13154,0.51,6577,2,1,1.0,0.0,0.0,0.0,...,1,6743,1,3371.50,6743,2,13154.0,0.0,0.0,0.0
67,55574853-48524952554875,30489,0.29,10163,3,3,1.0,0.0,0.0,0.0,...,0,8726,1,2908.67,8726,3,30489.0,0.0,0.0,0.0
80,55574957-51555751564974,12350,0.44,12350,1,1,0.0,0.0,0.0,0.0,...,0,5456,1,5456.00,5456,1,12350.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78615,55574851-54535057545771,17490,0.09,17490,1,1,1.0,0.0,0.0,0.0,...,1,1590,1,1590.00,1590,1,17490.0,0.0,0.0,0.0
78823,55575348-56555254505373,21778,0.20,21778,1,1,1.0,0.0,0.0,0.0,...,1,4355,1,4355.00,4355,1,21778.0,0.0,0.0,0.0
78878,55575048-51485548495070,13390,0.29,13390,1,1,1.0,0.0,0.0,0.0,...,1,3831,1,3831.00,3831,1,13390.0,0.0,0.0,0.0
79145,55575757-55564955555778,10989,0.23,10989,1,1,1.0,0.0,0.0,0.0,...,1,2539,1,2539.00,2539,1,10989.0,0.0,0.0,0.0


In [378]:
results = pd.DataFrame({
                        "XGBoost":[0]*6,

  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])
# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

final_iv, IV = data_vars(X_train.copy(), y_train.copy())

important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]
X = X[important_columns]    
depth=70

xgboost = XGBClassifier(random_state=0, max_depth = depth)
xgboost.fit(X_train, y_train)
train_pred = xgboost.predict(X_train)
test_pred = xgboost.predict(X_test)

predictions = xgboost.predict_proba(X)
print(predictions)

[[0.89998037 0.10001963]
 [0.7253673  0.2746327 ]
 [0.04680943 0.95319057]
 ...
 [0.05681461 0.9431854 ]
 [0.37286395 0.62713605]
 [0.18109101 0.818909  ]]


In [379]:
cluster_5['predictions'] = 0
cluster_5['predictions']=predictions
cluster_5

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
37,55575453-57485153485077,11990,0.39,11990,1,1,0.0,0.0,0.0,0.0,...,4651,1,4651.00,4651,1,11990.0,0.0,0.0,0.0,0.899980
43,55575351-57494852515273,12350,0.44,12350,1,1,0.0,0.0,0.0,0.0,...,5456,1,5456.00,5456,1,12350.0,0.0,0.0,0.0,0.725367
44,55574854-53495257575179,13154,0.51,6577,2,1,1.0,0.0,0.0,0.0,...,6743,1,3371.50,6743,2,13154.0,0.0,0.0,0.0,0.046809
67,55574853-48524952554875,30489,0.29,10163,3,3,1.0,0.0,0.0,0.0,...,8726,1,2908.67,8726,3,30489.0,0.0,0.0,0.0,0.044965
80,55574957-51555751564974,12350,0.44,12350,1,1,0.0,0.0,0.0,0.0,...,5456,1,5456.00,5456,1,12350.0,0.0,0.0,0.0,0.276109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78615,55574851-54535057545771,17490,0.09,17490,1,1,1.0,0.0,0.0,0.0,...,1590,1,1590.00,1590,1,17490.0,0.0,0.0,0.0,0.073389
78823,55575348-56555254505373,21778,0.20,21778,1,1,1.0,0.0,0.0,0.0,...,4355,1,4355.00,4355,1,21778.0,0.0,0.0,0.0,0.504421
78878,55575048-51485548495070,13390,0.29,13390,1,1,1.0,0.0,0.0,0.0,...,3831,1,3831.00,3831,1,13390.0,0.0,0.0,0.0,0.056815
79145,55575757-55564955555778,10989,0.23,10989,1,1,1.0,0.0,0.0,0.0,...,2539,1,2539.00,2539,1,10989.0,0.0,0.0,0.0,0.372864


### Cluster 6


In [380]:
cluster_6 = data[data['Labels'] == 6]

X = data[data['Labels']==6].drop(['target'],axis=1)
y = data[data['Labels']==6]['target']
cluster_6

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
2085,55575556-56524956574978,2269,0.31,206,10,11,0.0,0.0,0.0,0.0,...,0,706,1,70.60,706,11,0.0,0.0,1247.95,0.00
2466,55574956-53495351555374,8506,0.05,473,15,18,0.0,0.0,0.0,0.0,...,0,390,1,26.00,390,43,0.0,0.0,0.00,1446.02
2560,55575556-55535257495175,1897,0.29,474,4,4,0.0,0.0,0.0,0.0,...,0,556,1,139.00,556,4,0.0,0.0,948.50,948.50
8031,55575556-48555450525776,1665,0.15,333,5,5,0.0,0.0,0.0,0.0,...,0,257,1,51.40,257,5,0.0,0.0,1665.00,0.00
9396,55575556-55554952514971,6303,0.32,485,11,13,0.0,0.0,0.0,0.0,...,0,2022,1,183.82,2022,13,0.0,0.0,1449.69,3403.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74286,55575656-52495148545672,5280,0.06,5280,1,1,0.0,0.0,0.0,0.0,...,1,330,1,330.00,330,6,0.0,0.0,0.00,0.00
76738,55575556-56485149545275,980,0.29,980,1,1,0.0,0.0,0.0,0.0,...,1,280,1,280.00,280,1,980.0,0.0,0.00,0.00
76949,55574954-52575756495676,984,0.30,328,3,3,0.0,0.0,0.0,0.0,...,1,297,1,99.00,297,3,0.0,0.0,984.00,0.00
77156,55575556-56545256505277,1067,0.20,534,2,2,0.0,0.0,0.0,0.0,...,1,218,1,109.00,218,2,0.0,0.0,0.00,0.00


In [381]:
results = pd.DataFrame({
                        "CatBoost":[0]*6,

  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])
# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

final_iv, IV = data_vars(X_train.copy(), y_train.copy())

important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]
X = X[important_columns]    
estimators=200
cat_boost = CatBoostClassifier(random_state=609, verbose=0, n_estimators = estimators)
cat_boost.fit(X_train, y_train)
train_pred = cat_boost.predict(X_train)
test_pred = cat_boost.predict(X_test)

fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

predictions = cat_boost.predict_proba(X)
print(predictions)

[[0.72983389 0.27016611]
 [0.23846071 0.76153929]
 [0.16253094 0.83746906]
 [0.77772454 0.22227546]
 [0.76186046 0.23813954]
 [0.84497266 0.15502734]
 [0.60923183 0.39076817]
 [0.88537477 0.11462523]
 [0.39912054 0.60087946]
 [0.7056837  0.2943163 ]
 [0.67079902 0.32920098]
 [0.87056452 0.12943548]
 [0.53162354 0.46837646]
 [0.30540225 0.69459775]
 [0.19193261 0.80806739]
 [0.82201246 0.17798754]
 [0.89649764 0.10350236]
 [0.79864985 0.20135015]
 [0.17763725 0.82236275]
 [0.84210731 0.15789269]
 [0.8580293  0.1419707 ]
 [0.87399948 0.12600052]
 [0.81366173 0.18633827]
 [0.45691806 0.54308194]
 [0.84740496 0.15259504]
 [0.39608733 0.60391267]
 [0.64060235 0.35939765]
 [0.73414541 0.26585459]
 [0.8407395  0.1592605 ]
 [0.70150551 0.29849449]
 [0.7694843  0.2305157 ]
 [0.85636943 0.14363057]
 [0.84798939 0.15201061]
 [0.85613979 0.14386021]
 [0.52114969 0.47885031]
 [0.81176794 0.18823206]
 [0.82960412 0.17039588]
 [0.87484854 0.12515146]
 [0.85672295 0.14327705]
 [0.85911447 0.14088553]


In [382]:
cluster_6['predictions'] = 0
cluster_6['predictions']=predictions
cluster_6

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
2085,55575556-56524956574978,2269,0.31,206,10,11,0.0,0.0,0.0,0.0,...,706,1,70.60,706,11,0.0,0.0,1247.95,0.00,0.729834
2466,55574956-53495351555374,8506,0.05,473,15,18,0.0,0.0,0.0,0.0,...,390,1,26.00,390,43,0.0,0.0,0.00,1446.02,0.238461
2560,55575556-55535257495175,1897,0.29,474,4,4,0.0,0.0,0.0,0.0,...,556,1,139.00,556,4,0.0,0.0,948.50,948.50,0.162531
8031,55575556-48555450525776,1665,0.15,333,5,5,0.0,0.0,0.0,0.0,...,257,1,51.40,257,5,0.0,0.0,1665.00,0.00,0.777725
9396,55575556-55554952514971,6303,0.32,485,11,13,0.0,0.0,0.0,0.0,...,2022,1,183.82,2022,13,0.0,0.0,1449.69,3403.62,0.761860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74286,55575656-52495148545672,5280,0.06,5280,1,1,0.0,0.0,0.0,0.0,...,330,1,330.00,330,6,0.0,0.0,0.00,0.00,0.366502
76738,55575556-56485149545275,980,0.29,980,1,1,0.0,0.0,0.0,0.0,...,280,1,280.00,280,1,980.0,0.0,0.00,0.00,0.466609
76949,55574954-52575756495676,984,0.30,328,3,3,0.0,0.0,0.0,0.0,...,297,1,99.00,297,3,0.0,0.0,984.00,0.00,0.170644
77156,55575556-56545256505277,1067,0.20,534,2,2,0.0,0.0,0.0,0.0,...,218,1,109.00,218,2,0.0,0.0,0.00,0.00,0.711428


### Cluster 7

In [383]:
cluster_7 = data[data['Labels'] == 7]

X = data[data['Labels']==7].drop(['target'],axis=1)
y = data[data['Labels']==7]['target']
cluster_7

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
28,55574951-54505748494972,2350,0.54,2350,1,1,0.0,0.0,0.0,0.0,...,0,1271,1,1271.00,1271,1,2350.00,0.0,0.0,0.00
32,55574850-57525548544970,2927,0.42,732,4,4,0.0,0.0,0.0,0.0,...,0,1221,1,305.25,1221,4,0.00,0.0,0.0,2927.00
93,55575348-52515057505076,2467,0.55,2467,1,1,0.0,0.0,0.0,0.0,...,0,1346,1,1346.00,1346,1,2467.00,0.0,0.0,0.00
105,55574951-55575755555075,3793,0.38,1264,3,3,0.0,0.0,0.0,0.0,...,0,1425,1,475.00,1425,3,2541.31,0.0,0.0,1251.69
145,55574951-56515151565379,9290,0.44,9290,1,1,0.0,0.0,0.0,0.0,...,0,4114,1,4114.00,4114,1,9290.00,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79280,55575052-55485148515173,1304,0.02,261,3,5,0.0,0.0,0.0,0.0,...,1,28,1,9.33,28,10,0.00,0.0,0.0,0.00
79295,55574951-54535448484976,3449,0.17,3449,1,1,0.0,0.0,0.0,0.0,...,1,578,1,578.00,578,1,0.00,0.0,3449.0,0.00
79304,55574854-57525657555175,2679,0.23,2679,1,1,0.0,0.0,0.0,0.0,...,1,618,1,618.00,618,1,0.00,0.0,2679.0,0.00
79367,55574854-49575655495574,189,0.22,189,1,1,0.0,0.0,0.0,0.0,...,1,42,1,42.00,42,1,0.00,0.0,0.0,0.00


In [384]:
results = pd.DataFrame({
                        "XGBoost":[0]*6,

  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])
# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

final_iv, IV = data_vars(X_train.copy(), y_train.copy())

important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]
X = X[important_columns]    
depth=70

xgboost = XGBClassifier(random_state=0, max_depth = depth)
xgboost.fit(X_train, y_train)
train_pred = xgboost.predict(X_train)
test_pred = xgboost.predict(X_test)

predictions = xgboost.predict_proba(X)
print(predictions)

[[0.984923   0.01507699]
 [0.97103333 0.02896664]
 [0.98854387 0.01145614]
 ...
 [0.05242431 0.9475757 ]
 [0.02413678 0.9758632 ]
 [0.02570814 0.97429186]]


In [385]:
cluster_7['predictions'] = 0
cluster_7['predictions']=predictions
cluster_7

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
28,55574951-54505748494972,2350,0.54,2350,1,1,0.0,0.0,0.0,0.0,...,1271,1,1271.00,1271,1,2350.00,0.0,0.0,0.00,0.984923
32,55574850-57525548544970,2927,0.42,732,4,4,0.0,0.0,0.0,0.0,...,1221,1,305.25,1221,4,0.00,0.0,0.0,2927.00,0.971033
93,55575348-52515057505076,2467,0.55,2467,1,1,0.0,0.0,0.0,0.0,...,1346,1,1346.00,1346,1,2467.00,0.0,0.0,0.00,0.988544
105,55574951-55575755555075,3793,0.38,1264,3,3,0.0,0.0,0.0,0.0,...,1425,1,475.00,1425,3,2541.31,0.0,0.0,1251.69,0.722696
145,55574951-56515151565379,9290,0.44,9290,1,1,0.0,0.0,0.0,0.0,...,4114,1,4114.00,4114,1,9290.00,0.0,0.0,0.00,0.954843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79280,55575052-55485148515173,1304,0.02,261,3,5,0.0,0.0,0.0,0.0,...,28,1,9.33,28,10,0.00,0.0,0.0,0.00,0.100546
79295,55574951-54535448484976,3449,0.17,3449,1,1,0.0,0.0,0.0,0.0,...,578,1,578.00,578,1,0.00,0.0,3449.0,0.00,0.020853
79304,55574854-57525657555175,2679,0.23,2679,1,1,0.0,0.0,0.0,0.0,...,618,1,618.00,618,1,0.00,0.0,2679.0,0.00,0.052424
79367,55574854-49575655495574,189,0.22,189,1,1,0.0,0.0,0.0,0.0,...,42,1,42.00,42,1,0.00,0.0,0.0,0.00,0.024137


### Cluster 8

In [386]:
cluster_8 = data[data['Labels'] == 8]

X = data[data['Labels']==8].drop(['target'],axis=1)
y = data[data['Labels']==8]['target']
cluster_8

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
3725,55574854-52575248485270,1605,0.32,321,4,5,0.0,0.0,0.0,0.0,...,0,518,1,129.50,518,5,0.0,0.0,0.00,1605.00
3779,55575656-55515549515075,1548,0.42,516,2,3,0.0,0.0,0.0,0.0,...,0,654,1,327.00,654,3,0.0,0.0,0.00,1548.00
6924,55575056-48495250534874,11537,0.26,939,11,6,0.0,0.0,0.0,0.0,...,0,3047,1,277.00,3047,12,0.0,0.0,1961.29,9575.71
7336,55575056-56565457555670,4737,0.23,790,5,6,0.0,0.0,0.0,0.0,...,0,1084,1,216.80,1084,6,0.0,0.0,2368.50,2368.50
9077,55575056-54514948505571,2521,0.28,504,5,5,0.0,0.0,0.0,0.0,...,0,702,1,140.40,702,5,0.0,0.0,2521.00,0.00
12524,55575452-48515351485070,1532,0.04,511,3,3,0.0,0.0,0.0,0.0,...,0,62,1,20.67,62,3,0.0,0.0,0.00,0.00
13401,55575450-52535356535374,7140,0.16,7140,1,1,0.0,0.0,0.0,0.0,...,0,1140,1,1140.00,1140,6,0.0,0.0,0.00,0.00
13657,55574851-49505048535076,858,0.16,429,2,2,0.0,0.0,0.0,0.0,...,0,136,1,68.00,136,2,0.0,0.0,0.00,0.00
15379,55575449-52575250545070,2039,0.13,2039,1,1,0.0,0.0,0.0,0.0,...,0,274,1,274.00,274,1,0.0,0.0,2039.00,0.00
16824,55575156-57485150485179,370,0.72,370,1,1,0.0,0.0,0.0,0.0,...,0,268,1,268.00,268,2,0.0,0.0,0.00,0.00


In [387]:
results = pd.DataFrame({
                        "CatBoost":[0]*6,

  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])
# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

final_iv, IV = data_vars(X_train.copy(), y_train.copy())

important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]
X = X[important_columns]    
estimators=200
cat_boost = CatBoostClassifier(random_state=609, verbose=0, n_estimators = estimators)
cat_boost.fit(X_train, y_train)
train_pred = cat_boost.predict(X_train)
test_pred = cat_boost.predict(X_test)

fpr_train, tpr_train, thresholds = roc_curve(y_train, train_pred)
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred)

predictions = cat_boost.predict_proba(X)
print(predictions)

[[0.8184269  0.1815731 ]
 [0.72823671 0.27176329]
 [0.59969053 0.40030947]
 [0.82122786 0.17877214]
 [0.84183753 0.15816247]
 [0.79289597 0.20710403]
 [0.8383332  0.1616668 ]
 [0.80232646 0.19767354]
 [0.88798353 0.11201647]
 [0.75557977 0.24442023]
 [0.89581784 0.10418216]
 [0.82887878 0.17112122]
 [0.80537169 0.19462831]
 [0.38923784 0.61076216]
 [0.88294674 0.11705326]
 [0.7124586  0.2875414 ]
 [0.87750087 0.12249913]
 [0.86620239 0.13379761]
 [0.84276324 0.15723676]
 [0.8950249  0.1049751 ]
 [0.84980092 0.15019908]
 [0.76996574 0.23003426]
 [0.74521806 0.25478194]
 [0.78600588 0.21399412]
 [0.78761218 0.21238782]
 [0.44363216 0.55636784]
 [0.84125718 0.15874282]
 [0.89000946 0.10999054]
 [0.19902268 0.80097732]
 [0.23243823 0.76756177]
 [0.23746611 0.76253389]
 [0.4488938  0.5511062 ]
 [0.87617266 0.12382734]
 [0.76800181 0.23199819]
 [0.19966055 0.80033945]
 [0.22957526 0.77042474]
 [0.40473925 0.59526075]
 [0.71312724 0.28687276]
 [0.24465564 0.75534436]
 [0.28971572 0.71028428]]

In [388]:
cluster_8['predictions'] = 0
cluster_8['predictions']=predictions
cluster_8

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
3725,55574854-52575248485270,1605,0.32,321,4,5,0.0,0.0,0.0,0.0,...,518,1,129.50,518,5,0.0,0.0,0.00,1605.00,0.818427
3779,55575656-55515549515075,1548,0.42,516,2,3,0.0,0.0,0.0,0.0,...,654,1,327.00,654,3,0.0,0.0,0.00,1548.00,0.728237
6924,55575056-48495250534874,11537,0.26,939,11,6,0.0,0.0,0.0,0.0,...,3047,1,277.00,3047,12,0.0,0.0,1961.29,9575.71,0.599691
7336,55575056-56565457555670,4737,0.23,790,5,6,0.0,0.0,0.0,0.0,...,1084,1,216.80,1084,6,0.0,0.0,2368.50,2368.50,0.821228
9077,55575056-54514948505571,2521,0.28,504,5,5,0.0,0.0,0.0,0.0,...,702,1,140.40,702,5,0.0,0.0,2521.00,0.00,0.841838
12524,55575452-48515351485070,1532,0.04,511,3,3,0.0,0.0,0.0,0.0,...,62,1,20.67,62,3,0.0,0.0,0.00,0.00,0.792896
13401,55575450-52535356535374,7140,0.16,7140,1,1,0.0,0.0,0.0,0.0,...,1140,1,1140.00,1140,6,0.0,0.0,0.00,0.00,0.838333
13657,55574851-49505048535076,858,0.16,429,2,2,0.0,0.0,0.0,0.0,...,136,1,68.00,136,2,0.0,0.0,0.00,0.00,0.802326
15379,55575449-52575250545070,2039,0.13,2039,1,1,0.0,0.0,0.0,0.0,...,274,1,274.00,274,1,0.0,0.0,2039.00,0.00,0.887984
16824,55575156-57485150485179,370,0.72,370,1,1,0.0,0.0,0.0,0.0,...,268,1,268.00,268,2,0.0,0.0,0.00,0.00,0.755580


### Cluster 9

In [389]:
cluster_9 = data[data['Labels'] == 9]

X = data[data['Labels']==9].drop(['target'],axis=1)
y = data[data['Labels']==9]['target']
cluster_9

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
0,55574954-50515053535277,992,0.44,318,3,2,1.0,0.0,0.0,0.0,...,1,435,1,145.00,435,3,327.36,0.0,327.36,327.36
2,55574954-56545048505170,4038,0.28,281,18,9,1.0,0.0,0.0,0.0,...,0,1132,1,62.89,1132,19,242.28,0.0,2907.36,0.00
26,55575048-48485255555577,1329,0.46,121,8,4,0.0,1.0,0.0,0.0,...,0,612,1,76.50,612,9,0.00,0.0,1182.81,146.19
30,55575055-54535256505671,11006,0.27,984,13,2,0.0,1.0,0.0,0.0,...,0,3025,1,232.69,3025,14,1650.90,0.0,5943.24,2531.38
46,55574851-53545354554978,250,0.23,250,1,1,1.0,0.0,0.0,0.0,...,0,58,1,58.00,58,1,0.00,0.0,250.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79349,55574954-49505449495679,1949,0.29,1949,1,1,1.0,0.0,0.0,0.0,...,1,563,1,563.00,563,1,0.00,0.0,1949.00,0.00
79365,55575453-50565056524979,1330,0.32,1330,1,1,1.0,0.0,0.0,0.0,...,1,431,1,431.00,431,1,0.00,0.0,1330.00,0.00
79372,55574954-51534953485072,2542,0.05,2542,1,1,1.0,0.0,0.0,0.0,...,1,121,1,121.00,121,1,0.00,0.0,2542.00,0.00
79384,55574949-57565751545277,1399,0.36,1399,1,1,0.0,0.0,1.0,0.0,...,1,500,1,500.00,500,1,0.00,0.0,1399.00,0.00


In [390]:
results = pd.DataFrame({
                        "RandomForestClassifier":[0]*6,

  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])
# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

final_iv, IV = data_vars(X_train.copy(), y_train.copy())

important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]
X = X[important_columns]    
depth=50
best_n_estimators = 200

clf_forest = RandomForestClassifier(random_state=609, n_estimators = best_n_estimators, max_depth = depth)
clf_forest.fit(X_train, y_train)
train_pred = clf_forest.predict(X_train)
test_pred = clf_forest.predict(X_test)

predictions = clf_forest.predict_proba(X)
print(predictions)

[[0.24333333 0.75666667]
 [0.89991071 0.10008929]
 [0.645      0.355     ]
 ...
 [0.07046225 0.92953775]
 [0.005      0.995     ]
 [0.4295     0.5705    ]]


In [391]:
cluster_9['predictions'] = 0
cluster_9['predictions']=predictions
cluster_9

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
0,55574954-50515053535277,992,0.44,318,3,2,1.0,0.0,0.0,0.0,...,435,1,145.00,435,3,327.36,0.0,327.36,327.36,0.243333
2,55574954-56545048505170,4038,0.28,281,18,9,1.0,0.0,0.0,0.0,...,1132,1,62.89,1132,19,242.28,0.0,2907.36,0.00,0.899911
26,55575048-48485255555577,1329,0.46,121,8,4,0.0,1.0,0.0,0.0,...,612,1,76.50,612,9,0.00,0.0,1182.81,146.19,0.645000
30,55575055-54535256505671,11006,0.27,984,13,2,0.0,1.0,0.0,0.0,...,3025,1,232.69,3025,14,1650.90,0.0,5943.24,2531.38,0.755000
46,55574851-53545354554978,250,0.23,250,1,1,1.0,0.0,0.0,0.0,...,58,1,58.00,58,1,0.00,0.0,250.00,0.00,0.995000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79349,55574954-49505449495679,1949,0.29,1949,1,1,1.0,0.0,0.0,0.0,...,563,1,563.00,563,1,0.00,0.0,1949.00,0.00,0.263205
79365,55575453-50565056524979,1330,0.32,1330,1,1,1.0,0.0,0.0,0.0,...,431,1,431.00,431,1,0.00,0.0,1330.00,0.00,0.350500
79372,55574954-51534953485072,2542,0.05,2542,1,1,1.0,0.0,0.0,0.0,...,121,1,121.00,121,1,0.00,0.0,2542.00,0.00,0.070462
79384,55574949-57565751545277,1399,0.36,1399,1,1,0.0,0.0,1.0,0.0,...,500,1,500.00,500,1,0.00,0.0,1399.00,0.00,0.005000


### Cluster 10

In [392]:
cluster_10 = data[data['Labels'] == 10]

X = data[data['Labels']==10].drop(['target'],axis=1)
y = data[data['Labels']==10]['target']
cluster_10

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
1358,55575052-53535752485076,6367,0.20,564,13,6,0.0,0.0,0.0,0.0,...,1,1301,1,100.08,1301,14,955.05,0.0,3947.54,509.36
1752,55574952-48535757574872,699,0.26,699,1,1,0.0,0.0,0.0,0.0,...,0,180,1,180.00,180,1,0.00,0.0,0.00,699.00
2004,55574850-53545154505272,10728,0.32,1300,7,3,0.0,0.0,0.0,0.0,...,1,3387,1,483.86,3387,8,1287.36,0.0,0.00,9440.64
2341,55574850-49545253555479,6846,0.23,2974,3,2,0.0,0.0,0.0,0.0,...,1,1567,1,522.33,1567,4,1711.50,0.0,0.00,5134.50
3288,55575651-53515457535777,6310,0.33,6310,1,1,0.0,0.0,0.0,0.0,...,0,2110,1,2110.00,2110,1,6310.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77902,55574952-56515349565274,2300,0.17,2300,1,1,0.0,0.0,0.0,0.0,...,1,400,1,400.00,400,2,0.00,0.0,0.00,0.00
78370,55574952-56525748535572,2000,0.04,2000,1,1,0.0,0.0,0.0,0.0,...,1,81,1,81.00,81,1,0.00,0.0,2000.00,0.00
78591,55574850-53545048525777,633,-0.04,633,1,1,0.0,0.0,0.0,0.0,...,1,-26,1,-26.00,-26,1,0.00,0.0,633.00,0.00
78734,55575651-52534850484875,1050,0.11,1050,1,1,0.0,0.0,0.0,0.0,...,1,120,1,120.00,120,1,0.00,0.0,0.00,0.00


In [393]:
results = pd.DataFrame({
                        "XGBoost":[0]*6,

  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])
# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

final_iv, IV = data_vars(X_train.copy(), y_train.copy())

important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]
X = X[important_columns]    
depth=70

xgboost = XGBClassifier(random_state=0, max_depth = depth)
xgboost.fit(X_train, y_train)
train_pred = xgboost.predict(X_train)
test_pred = xgboost.predict(X_test)

predictions = xgboost.predict_proba(X)
print(predictions)

[[0.217875   0.782125  ]
 [0.4967187  0.5032813 ]
 [0.08744508 0.9125549 ]
 [0.40067542 0.5993246 ]
 [0.9647575  0.03524249]
 [0.8944367  0.10556327]
 [0.9433732  0.05662678]
 [0.2528832  0.7471168 ]
 [0.89588207 0.10411792]
 [0.9743722  0.02562779]
 [0.98531216 0.01468782]
 [0.9835789  0.0164211 ]
 [0.99286914 0.00713087]
 [0.97941357 0.02058646]
 [0.9660996  0.03390041]
 [0.01201701 0.987983  ]
 [0.97435904 0.02564095]
 [0.07656348 0.9234365 ]
 [0.9447894  0.0552106 ]
 [0.96452373 0.03547626]
 [0.8949169  0.10508308]
 [0.7573777  0.24262233]
 [0.9236121  0.07638788]
 [0.9928597  0.00714026]
 [0.07380486 0.92619514]
 [0.9963421  0.0036579 ]
 [0.9693484  0.03065161]
 [0.12178588 0.8782141 ]
 [0.01440161 0.9855984 ]
 [0.01995248 0.9800475 ]
 [0.8794168  0.12058316]
 [0.7225562  0.27744383]
 [0.96671265 0.03328737]
 [0.9918104  0.00818963]
 [0.10202825 0.89797175]
 [0.93783736 0.06216262]
 [0.9499275  0.05007251]
 [0.96294564 0.03705434]
 [0.9550077  0.04499234]
 [0.143929   0.856071  ]


In [394]:
cluster_10['predictions'] = 0
cluster_10['predictions']=predictions
cluster_10

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
1358,55575052-53535752485076,6367,0.20,564,13,6,0.0,0.0,0.0,0.0,...,1301,1,100.08,1301,14,955.05,0.0,3947.54,509.36,0.217875
1752,55574952-48535757574872,699,0.26,699,1,1,0.0,0.0,0.0,0.0,...,180,1,180.00,180,1,0.00,0.0,0.00,699.00,0.496719
2004,55574850-53545154505272,10728,0.32,1300,7,3,0.0,0.0,0.0,0.0,...,3387,1,483.86,3387,8,1287.36,0.0,0.00,9440.64,0.087445
2341,55574850-49545253555479,6846,0.23,2974,3,2,0.0,0.0,0.0,0.0,...,1567,1,522.33,1567,4,1711.50,0.0,0.00,5134.50,0.400675
3288,55575651-53515457535777,6310,0.33,6310,1,1,0.0,0.0,0.0,0.0,...,2110,1,2110.00,2110,1,6310.00,0.0,0.00,0.00,0.964758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77902,55574952-56515349565274,2300,0.17,2300,1,1,0.0,0.0,0.0,0.0,...,400,1,400.00,400,2,0.00,0.0,0.00,0.00,0.020255
78370,55574952-56525748535572,2000,0.04,2000,1,1,0.0,0.0,0.0,0.0,...,81,1,81.00,81,1,0.00,0.0,2000.00,0.00,0.031413
78591,55574850-53545048525777,633,-0.04,633,1,1,0.0,0.0,0.0,0.0,...,-26,1,-26.00,-26,1,0.00,0.0,633.00,0.00,0.046871
78734,55575651-52534850484875,1050,0.11,1050,1,1,0.0,0.0,0.0,0.0,...,120,1,120.00,120,1,0.00,0.0,0.00,0.00,0.061873


### Cluster 11

In [395]:
cluster_11 = data[data['Labels'] == 11]

X = data[data['Labels']==11].drop(['target'],axis=1)
y = data[data['Labels']==11]['target']
cluster_11

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
113,55575054-55515754485678,7595,0.17,1615,6,2,1.0,0.0,0.0,0.0,...,0,1315,1,219.17,1315,7,0.00,2202.55,1063.30,4329.15
709,55574851-53545757525678,8567,0.14,710,12,3,1.0,0.0,0.0,0.0,...,1,1191,1,99.25,1191,16,685.36,2655.77,1285.05,0.00
844,55575049-56565148574978,3210,0.23,882,11,9,1.0,0.0,0.0,0.0,...,0,741,1,67.36,741,18,0.00,2182.80,0.00,160.50
981,55574954-50575148554871,7646,0.24,1274,6,6,1.0,0.0,0.0,0.0,...,0,1845,1,307.50,1845,6,0.00,2523.18,5122.82,0.00
1329,55575054-53515251564971,8591,0.11,616,35,8,1.0,0.0,0.0,0.0,...,1,974,1,27.83,974,64,429.55,7560.08,343.64,343.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79228,55574955-51575252484877,375,0.03,125,2,3,0.0,1.0,0.0,0.0,...,1,11,1,5.50,11,5,0.00,375.00,0.00,0.00
79230,55574851-53524955525476,129,0.47,129,1,1,1.0,0.0,0.0,0.0,...,1,61,1,61.00,61,1,0.00,129.00,0.00,0.00
79234,55575148-52485054515070,2065,0.18,516,2,4,1.0,0.0,0.0,0.0,...,1,367,1,183.50,367,35,0.00,2065.00,0.00,0.00
79235,55575054-57544952574879,4897,-0.08,377,3,13,1.0,0.0,0.0,0.0,...,1,-381,1,-127.00,-381,103,0.00,4897.00,0.00,0.00


In [396]:
results = pd.DataFrame({
                        "RandomForestClassifier":[0]*6,

  },index=["ROC-AUC Train","ROC-AUC Test",'F1 Train','F1 Test','GINI Train','GINI Test'])
# normalization process
numeric_data = X.select_dtypes([np.number]).copy()
scaler = StandardScaler()
numeric_data[numeric_data.columns] = scaler.fit_transform(numeric_data[numeric_data.columns])

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=609, stratify=y) 
X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5, random_state=609, stratify=y)

final_iv, IV = data_vars(X_train.copy(), y_train.copy())

important_columns = np.array(IV[(IV['IV'] >= 0.01)]['VAR_NAME'])

X_train = X_train[important_columns]
X_test = X_test[important_columns]
X_valid = X_valid[important_columns]
X = X[important_columns]    
depth=50
best_n_estimators = 200

clf_forest = RandomForestClassifier(random_state=609, n_estimators = best_n_estimators, max_depth = depth)
clf_forest.fit(X_train, y_train)
train_pred = clf_forest.predict(X_train)
test_pred = clf_forest.predict(X_test)

predictions = clf_forest.predict_proba(X)
print(predictions)

[[0.285   0.715  ]
 [0.25    0.75   ]
 [0.73    0.27   ]
 ...
 [0.46875 0.53125]
 [0.135   0.865  ]
 [0.31    0.69   ]]


In [397]:
cluster_11['predictions'] = 0
cluster_11['predictions']=predictions
cluster_11

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
113,55575054-55515754485678,7595,0.17,1615,6,2,1.0,0.0,0.0,0.0,...,1315,1,219.17,1315,7,0.00,2202.55,1063.30,4329.15,0.28500
709,55574851-53545757525678,8567,0.14,710,12,3,1.0,0.0,0.0,0.0,...,1191,1,99.25,1191,16,685.36,2655.77,1285.05,0.00,0.25000
844,55575049-56565148574978,3210,0.23,882,11,9,1.0,0.0,0.0,0.0,...,741,1,67.36,741,18,0.00,2182.80,0.00,160.50,0.73000
981,55574954-50575148554871,7646,0.24,1274,6,6,1.0,0.0,0.0,0.0,...,1845,1,307.50,1845,6,0.00,2523.18,5122.82,0.00,0.47500
1329,55575054-53515251564971,8591,0.11,616,35,8,1.0,0.0,0.0,0.0,...,974,1,27.83,974,64,429.55,7560.08,343.64,343.64,0.15000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79228,55574955-51575252484877,375,0.03,125,2,3,0.0,1.0,0.0,0.0,...,11,1,5.50,11,5,0.00,375.00,0.00,0.00,0.19500
79230,55574851-53524955525476,129,0.47,129,1,1,1.0,0.0,0.0,0.0,...,61,1,61.00,61,1,0.00,129.00,0.00,0.00,0.24500
79234,55575148-52485054515070,2065,0.18,516,2,4,1.0,0.0,0.0,0.0,...,367,1,183.50,367,35,0.00,2065.00,0.00,0.00,0.46875
79235,55575054-57544952574879,4897,-0.08,377,3,13,1.0,0.0,0.0,0.0,...,-381,1,-127.00,-381,103,0.00,4897.00,0.00,0.00,0.13500


## Final dataset with responses

In [398]:
clusters = [cluster_0, cluster_1, cluster_2, cluster_3, cluster_4, cluster_5, cluster_6, cluster_7, cluster_8, cluster_9, cluster_10, cluster_11]  

merged_df = pd.concat(datasets, axis=0, ignore_index=True)


merged_df.tail(50)

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,predictions
79339,55574852-52575754545776,1165,0.01,1165,1,1,0.0,0.0,0.0,0.0,...,6,1,6.00,6,1,0.0,1165.0,0.0,0.0,0.417810
79340,55574954-50485151495671,2999,-0.43,2999,1,1,1.0,0.0,0.0,0.0,...,-1291,1,-1291.00,-1291,1,0.0,2999.0,0.0,0.0,0.155000
79341,55574954-56525349525074,299,0.26,299,1,1,1.0,0.0,0.0,0.0,...,77,1,77.00,77,1,0.0,299.0,0.0,0.0,0.513458
79342,55574948-50524956534972,1049,0.02,1049,1,1,1.0,0.0,0.0,0.0,...,19,1,19.00,19,1,0.0,1049.0,0.0,0.0,0.213905
79343,55574853-55495457535276,829,0.01,829,1,1,1.0,0.0,0.0,0.0,...,9,1,9.00,9,1,0.0,829.0,0.0,0.0,0.120000
79344,55575054-57575356495071,1992,0.18,1992,1,1,1.0,0.0,0.0,0.0,...,365,1,365.00,365,24,0.0,1992.0,0.0,0.0,0.710000
79345,55574954-53575255514871,799,0.09,799,1,1,1.0,0.0,0.0,0.0,...,72,1,72.00,72,1,0.0,799.0,0.0,0.0,0.940000
79346,55574948-52555751555270,1125,0.02,281,4,4,1.0,0.0,0.0,0.0,...,24,1,6.00,24,14,0.0,1125.0,0.0,0.0,0.175000
79347,55574954-53545150535673,570,-0.35,95,1,6,1.0,0.0,0.0,0.0,...,-197,1,-197.00,-197,30,0.0,570.0,0.0,0.0,0.110000
79348,55574854-56554848554870,1487,-0.01,496,3,3,0.0,0.0,0.0,0.0,...,-8,1,-2.67,-8,3,0.0,1487.0,0.0,0.0,0.437500


In [401]:
merged_df.rename(columns={"predictions": "Response probability"}, inplace = True)
merged_df = merged_df
merged_df

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка,Response probability
0,55575657-55485751525479,12464,0.24,4155,3,3,0.0,0.0,0.0,1.0,...,2992,1,997.33,2992,3,12464.0,0.0,0.0,0.0,0.064895
1,55574857-51574948505274,16434,0.39,3689,5,2,0.0,0.0,0.0,1.0,...,6359,1,1271.80,6359,5,13147.2,0.0,3286.8,0.0,0.571591
2,55574857-52525255485279,3040,0.43,3040,1,1,0.0,0.0,0.0,1.0,...,1314,1,1314.00,1314,1,3040.0,0.0,0.0,0.0,0.945499
3,55574852-52525753535670,2692,0.68,538,3,5,0.0,0.0,0.0,1.0,...,1842,1,614.00,1842,8,0.0,0.0,0.0,2692.0,0.979691
4,55575056-52485553515670,1358,0.39,340,4,4,0.0,0.0,0.0,1.0,...,531,1,132.75,531,4,0.0,0.0,1358.0,0.0,0.912032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79384,55574955-51575252484877,375,0.03,125,2,3,0.0,1.0,0.0,0.0,...,11,1,5.50,11,5,0.0,375.0,0.0,0.0,0.195000
79385,55574851-53524955525476,129,0.47,129,1,1,1.0,0.0,0.0,0.0,...,61,1,61.00,61,1,0.0,129.0,0.0,0.0,0.245000
79386,55575148-52485054515070,2065,0.18,516,2,4,1.0,0.0,0.0,0.0,...,367,1,183.50,367,35,0.0,2065.0,0.0,0.0,0.468750
79387,55575054-57544952574879,4897,-0.08,377,3,13,1.0,0.0,0.0,0.0,...,-381,1,-127.00,-381,103,0.0,4897.0,0.0,0.0,0.135000


In [403]:
merged_df.columns

Index(['Телефон_new', 'Выручка_bought', 'Маржа_bought', 'Средний чек_bought',
       'Кол-во чеков_bought', 'Среднее кол-во товаров в  чеке_bought',
       'Центральный ф.о.', 'Приволжский ф.о.', 'Северо-Западный ф.о.',
       'Южный ф.о.', 'Сибирский ф.о.', 'Уральский ф.о.',
       'Дальневосточный ф.о.', 'Восточно-Сибирский ф.о.', 'Крымский ф.о.',
       'Северо-Кавказский ф.о.', 'КГТ', 'ОДЕЖДА', 'ИГРУШКИт', 'ППКП', 'ИНОЕ',
       'КРУПНОГАБАРИТНЫЙ ТОВАР выручка', 'ТЕКСТИЛЬ, ТРИКОТАЖ выручка',
       'ОБУВЬ выручка', 'КАНЦТОВАРЫ, КНИГИ, ДИСКИ выручка',
       'ТОВАРЫ ДЛЯ КОРМЛЕНИЯ выручка', 'ИГРУШКИ выручка',
       'КОСМЕТИКА/ГИГИЕНА выручка', 'СОПУТСТВУЮЩИЕ ТОВАРЫ выручка',
       'ЖЕНСКИЕ ШТУЧКИ выручка', 'ПОДГУЗНИКИ выручка',
       'ДЕТСКОЕ ПИТАНИЕ выручка', 'ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА выручка',
       'ТОВАРЫ ДЛЯ ЖИВОТНЫХ выручка', 'ППКП выручка', 'Labels', 'target',
       'Ценность клиента_bought', 'Кол-во уникальных клиентов_bought',
       'Средняя маржа_bought', 'Абсолютная

In [ ]:
merged_df.to_csv('response_table.csv',index=False,encoding='utf-8')

## Mean response probability by clusters

In [9]:
import pandas as pd

df = pd.read_csv('response_table.csv')


# Calculate the mean response probability per cluster
mean_response = df.groupby('Labels')['Response probability'].mean() * 100

# Create a new DataFrame to store the cluster, segment label, and mean response probability
result_df = pd.DataFrame({'Cluster': mean_response.index, 'Mean Response Probability': mean_response.values})

# Format the 'Mean Response Probability' column as percentages with two decimal places
result_df['Mean Response Probability'] = result_df['Mean Response Probability'].round(2).astype(str) + '%'

# Add the 'Segment' label to the 'Cluster' column
result_df['Cluster'] = 'Segment ' + result_df['Cluster'].astype(str)

# Save the resulting dataset as a CSV file
result_df.to_csv('cluster_mean_response.csv', index=False)



In [10]:
result_df

,Cluster,Mean Response Probability
0,Segment 0,49.03%
1,Segment 1,49.55%
2,Segment 2,53.56%
3,Segment 3,29.78%
4,Segment 4,48.49%
5,Segment 5,52.25%
6,Segment 6,53.57%
7,Segment 7,54.86%
8,Segment 8,67.1%
9,Segment 9,49.43%
